# Índice de Entradas

1. [General Statistics](#1)
2. [Boxplots](#2)
3. [Integral Helicidad](#3)
4. [Integral Helicidad Absoluta](#4)
5. [Integral Enstrofia](#5)
6. [3D Curl](#6)
7. [Modulo Rotacional](#7)
8. [Histograma Derivadas](#8)
9. [Comparaciones Laseres](#9)


@siva se separan las celdas para indicar la variable root_folder

In [22]:
root_folder = r"D:\Code\Hypatia\Fechas\2025\25Ene25"

In [ ]:
import os
import re

def buscar_labs_en_carpetas(root_folder):
    # Expresión regular para encontrar laboratorios en formato "Lab_algo" o "LabAlgo"
    patron = re.compile(r"(Lab(?:_|)\w+)$")
    labs_encontrados = []

    for carpeta in os.listdir(root_folder):
        ruta_completa = os.path.join(root_folder, carpeta)
        if os.path.isdir(ruta_completa):
            # Buscar coincidencia con la expresión regular
            coincidencia = patron.search(carpeta)
            if coincidencia:
                # Guardar el nombre del laboratorio tal como se encuentra
                labs_encontrados.append(coincidencia.group(1))

    return labs_encontrados

labs = buscar_labs_en_carpetas(root_folder)
print("Laboratorios encontrados:", labs)

if len(labs) > 1:
    chooser = True
else:
    chooser = False

triad_names = {
    "FRT": "Front-Right-Top",  
    "PLB": "Rear-Left-Bottom",  
    "FLT": "Front-Left-Top",  
    "PRB": "Rear-Right-Bottom",  
    "FRB": "Front-Right-Bottom",  
    "PLT": "Rear-Left-Top",  
    "FLB": "Front-Left-Bottom",  
    "PRT": "Rear-Right-Top",  
    "RTB": "Right-Top-Bottom",  
    "FLP": "Front-Left-Rear",  
    "LTB": "Left-Top-Bottom",  
    "FRP": "Front-Right-Rear"
}

triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

@siva esta celda contiene listas, diccionarios y variables necesarios para correr cada una de las celdas que siguen. Se añade la variable chooser la cual es un booleano el cual nos permitirá determinar la forma de proceder al generar las gráficas.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew, iqr
from matplotlib.backends.backend_pdf import PdfPages
import openpyxl
from datetime import datetime

# Función para leer datos del sensor desde un archivo
def read_sensor_data(filepath):
    data = []
    timestamps = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except UnicodeDecodeError:
        print(f"Unicode decode error in file {filepath}")
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return timestamps, np.array(data).T

# Función para analizar el contenido de una carpeta
def analyze_folder(root_folder):
    data_per_file = {}
    for root, dirs, files in os.walk(root_folder):
        # Evitar carpetas llamadas "Data Analysis"
        if 'Data Analysis' in dirs:
            dirs.remove('Data Analysis')
            
        for filename in files:
            if filename.startswith("0") and filename.endswith(("negra.txt", "roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt","naranja.txt")):
                file_path = os.path.join(root, filename)
                timestamps, data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = (timestamps, data, filename)
    return data_per_file

# Función para generar estadísticas de los datos
def generate_statistics(data):
    return {
        'mean': np.mean(data, axis=1),
        'median': np.median(data, axis=1),
        'variance': np.var(data, axis=1),
        'stdDev': np.std(data, axis=1),
        'min': np.min(data, axis=1),
        'max': np.max(data, axis=1),
        'firstQuartile': np.percentile(data, 25, axis=1),
        'thirdQuartile': np.percentile(data, 75, axis=1),
        'iqr': iqr(data, axis=1),
        'trimmedVariance': [np.var(np.trim_zeros(np.sort(sensor_data))) for sensor_data in data],
        'kurtosis': kurtosis(data, axis=1, fisher=True, bias=False),
        'skewness': skew(data, axis=1, bias=False)
    }

# Función para extraer la fecha del directorio root_folder
def extract_date_from_root_folder(root_folder):
    return os.path.basename(root_folder)

# Función para formatear las estadísticas
def format_statistics(stats, fmt=".10f"):
    return {key: [f"{val:{fmt}}" for val in values] for key, values in stats.items()}

# Función para asignar colores basados en el sufijo del archivo
def get_color_for_file(filename):
    if "negra" in filename:
        return "black"
    elif "roja" in filename:
        return "red"
    elif "morada" in filename:
        return "purple"
    elif "azul" in filename:
        return "blue"
    elif "verde" in filename:
        return "green"
    elif "amarilla" in filename:
        return "yellow"
    else:
        return "grey"

# Función para generar los gráficos y los PDFs
def plot_comparison_grid(data_per_file, output_pdf_folder, output_img_folder, output_text_folder, lab, session, date):
    component_names = ["FRONT(A0)", "REAR(A1)", "LEFT(A2)", "RIGHT(A3)", "TOP(A4)", "BOTTOM(A5)"]
    session_name = "Afternoon" if session == "V" else "Morning"

    for filename, (timestamps, data, file_path) in data_per_file.items():
        color = get_color_for_file(filename)  # Color basado en el sufijo del archivo

        # Determinar las subcarpetas Baseline o Experimental Color
        if "mednegra.txt" in filename:
            subfolder = "Baseline"
        else:
            subfolder = "Experimental Color"

        pdf_file = os.path.join(output_pdf_folder, subfolder, f"{filename}.pdf")
        img_file = os.path.join(output_img_folder, subfolder, f"{filename}.png")

        with PdfPages(pdf_file) as pdf:
            if lab in labs:
                n_lab = lab
            fig, axes = plt.subplots(6, 3, figsize=(36, 45))  # Tamaño más grande de la figura
            fig.subplots_adjust(top=0.85, wspace=0.6)  # Más espacio entre columnas, se puede modificar wspace para ajustar el espacio
            fig.suptitle(f"Análisis del archivo: {filename}\n"
                         f"Laboratorio: {n_lab}, Turno: {session_name}, Fecha: {date}", fontsize=40, y=0.98)  # Título más grande
            axes = axes.flatten()
            stats = generate_statistics(data)  # Generar estadísticas una vez por archivo
            stats_formatted = format_statistics(stats, fmt=".10f")

            for i in range(6):
                # Gráfico de Datos Crudos
                ax = axes[3 * i]
                ax.plot(data[i], label='Datos', color=color)
                ax.set_title(f"Component {component_names[i]} - Raw data", fontsize=27)  # Títulos más grandes
                ax.set_xlabel("Registers", fontsize=25)
                ax.set_ylabel("Volts", fontsize=25)
                ax.legend()
                ax.tick_params(axis='x', rotation=45, labelsize=25)  # Tamaño de las etiquetas de los ejes x y y
                ax.tick_params(axis='y', labelsize=25)  # Tamaño de las etiquetas de los ejes y

                # Histograma de Derivadas
                derivative = np.gradient(data[i])
                ax_hist = axes[3 * i + 1]
                ax_hist.hist(derivative, bins=25, color=color, edgecolor='black')
                ax_hist.set_title(f"Component {component_names[i]} - Derivative Histograms", fontsize=27)
                ax_hist.set_xlabel("Derivative Value", fontsize=25)
                ax_hist.set_ylabel("Frequence", fontsize=25)
                ax_hist.tick_params(axis='x', labelsize=25)  # Tamaño de las etiquetas de los ejes x
                ax_hist.tick_params(axis='y', labelsize=25)  # Tamaño de las etiquetas de los ejes y

                # Texto de Estadísticas de Datos Crudos
                stats_text = (f"Mean: {stats_formatted['mean'][i]}\n"
                              f"Median: {stats_formatted['median'][i]}\n"
                              f"Variance: {stats_formatted['variance'][i]}\n"
                              f"Standard Deviation: {stats_formatted['stdDev'][i]}\n"
                              f"Minimun: {stats_formatted['min'][i]}\n"
                              f"Maximun: {stats_formatted['max'][i]}\n"
                              f"First Quartile: {stats_formatted['firstQuartile'][i]}\n"
                              f"Third Cuartile: {stats_formatted['thirdQuartile'][i]}\n"
                              f"IQR: {stats_formatted['iqr'][i]}\n"
                              f"Trimmed Variance: {stats_formatted['trimmedVariance'][i]}\n"
                              f"Kurtosis: {stats_formatted['kurtosis'][i]}\n"
                              f"Skewness: {stats_formatted['skewness'][i]}")
                ax_stats = axes[3 * i + 2]
                ax_stats.text(0.1, 0.5, stats_text, fontsize=25, verticalalignment='center')
                ax_stats.axis('off')
                ax_stats.set_title(f"Component {component_names[i]} - Raw Data Satistics", fontsize=24)

            plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajusta el espacio de toda la figura
            pdf.savefig(fig)  # Guarda el gráfico en el PDF
            plt.savefig(img_file, dpi=400)  # Guarda el gráfico como imagen de alta calidad
            plt.grid(True)
            plt.close()

        # Generar archivo de texto con las estadísticas
        text_folder = os.path.join(output_text_folder, subfolder)
        stats_file = os.path.join(text_folder, f"{filename}.txt")
        os.makedirs(os.path.dirname(stats_file), exist_ok=True)
        with open(stats_file, 'w', encoding='utf-8') as f:
            f.write(f"{date},{session_name},{filename}\n\n")
            for i, component in enumerate(component_names):
                f.write(f"{component}\n")
                f.write(f"{stats_formatted['median'][i]},{stats_formatted['mean'][i]},{stats_formatted['variance'][i]},{stats_formatted['stdDev'][i]},{stats_formatted['min'][i]},{stats_formatted['max'][i]},{stats_formatted['firstQuartile'][i]},{stats_formatted['thirdQuartile'][i]},{stats_formatted['iqr'][i]},{stats_formatted['trimmedVariance'][i]},{stats_formatted['kurtosis'][i]},{stats_formatted['skewness'][i]}\n\n")

# Función para añadir datos al Excel
def add_data_to_excel(sheet, stats_formatted, date, session_name, filename, lab):
    component_names = ["FRONT(A0)", "REAR(A1)", "LEFT(A2)", "RIGHT(A3)", "TOP(A4)", "BOTTOM(A5)"]
    row_data = [date, session_name, lab, filename]  # Incluyendo el nombre del laboratorio

    for i in range(6):
        row_data.extend([
            stats_formatted['median'][i], stats_formatted['mean'][i], stats_formatted['variance'][i],
            stats_formatted['stdDev'][i], stats_formatted['min'][i], stats_formatted['max'][i],
            stats_formatted['firstQuartile'][i], stats_formatted['thirdQuartile'][i], stats_formatted['iqr'][i],
            stats_formatted['trimmedVariance'][i], stats_formatted['kurtosis'][i], stats_formatted['skewness'][i]
        ])
        row_data.append("")  # Espacio entre componentes

    sheet.append(row_data)

def process_all_experiments(root_folder):
    date = extract_date_from_root_folder(root_folder)
    date = date.replace("- copia", "").strip()
    sessions = ["V", "M"]

    # Creando un nuevo libro de trabajo y hoja
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Datos"

    # Añadiendo títulos
    titles = [
        "date", "shift", "lab", "filename",
        "Median_front", "Mean_front", "Variance_front", "STD_front", "Min_front", "Max_front", "First Quartile_front", "Third Quartile_front", "IQR_front", "Trimmed Variance (0.2)_front", "Kurtosis_front", "Skewness_front", "",
        "Median_rear", "Mean_rear", "Variance_rear", "STD_rear", "Min_rear", "Max_rear", "First Quartile_rear", "Third Quartile_rear", "IQR_rear", "Trimmed Variance (0.2)_rear", "Kurtosis_rear", "Skewness_rear", "",
        "Median_left", "Mean_left", "Variance_left", "STD_left", "Min_left", "Max_left", "First Quartile_left", "Third Quartile_left", "IQR_left", "Trimmed Variance (0.2)_left", "Kurtosis_left", "Skewness_left", "",
        "Median_right", "Mean_right", "Variance_right", "STD_right", "Min_right", "Max_right", "First Quartile_right", "Third Quartile_right", "IQR_right", "Trimmed Variance (0.2)_right", "Kurtosis_right", "Skewness_right", "",
        "Median_top", "Mean_top", "Variance_top", "STD_top", "Min_top", "Max_top", "First Quartile_top", "Third Quartile_top", "IQR_top", "Trimmed Variance (0.2)_top", "Kurtosis_top", "Skewness_top", "",
        "Median_bottom", "Mean_bottom", "Variance_bottom", "STD_bottom", "Min_bottom", "Max_bottom", "First Quartile_bottom", "Third Quartile_bottom", "IQR_bottom", "Trimmed Variance (0.2)_bottom", "Kurtosis_bottom", "Skewness_bottom", ""
    ]
    sheet.append(titles)

    all_data = []  # Lista para almacenar todos los datos



    for lab in labs:
        for session in sessions:
            experiment_folder = os.path.join(root_folder, f"{date} - {lab}", f"{date}.{session} - {lab}")
            print(f"Checking directory: {experiment_folder}")
            if os.path.exists(experiment_folder):
                output_pdf_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "General Statistics", "PDFs")
                output_img_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "General Statistics", "Images")
                output_text_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "General Statistics Text")
                os.makedirs(output_pdf_folder, exist_ok=True)
                os.makedirs(output_img_folder, exist_ok=True)
                os.makedirs(output_text_folder, exist_ok=True)

                # Crear subcarpetas Baseline y Experimental Color
                os.makedirs(os.path.join(output_pdf_folder, "Baseline"), exist_ok=True)
                os.makedirs(os.path.join(output_pdf_folder, "Experimental Color"), exist_ok=True)
                os.makedirs(os.path.join(output_img_folder, "Baseline"), exist_ok=True)
                os.makedirs(os.path.join(output_img_folder, "Experimental Color"), exist_ok=True)
                os.makedirs(os.path.join(output_text_folder, "Baseline"), exist_ok=True)
                os.makedirs(os.path.join(output_text_folder, "Experimental Color"), exist_ok=True)

                data_per_file = analyze_folder(experiment_folder)
                session_name = "Vespertino" if session == "V" else "Matutino"

                for filename, (timestamps, data, file_path) in data_per_file.items():
                    stats = generate_statistics(data)
                    stats_formatted = format_statistics(stats, fmt=".10f")
                    # Almacenar los datos en una lista para ordenarlos después
                    all_data.append({
                        'date': date,
                        'session_name': session_name,
                        'lab': lab,
                        'filename': filename,
                        'stats_formatted': stats_formatted
                    })

                # Llamar a la función de gráficos
                plot_comparison_grid(data_per_file, output_pdf_folder, output_img_folder, output_text_folder, lab, session, date)
            else:
                print(f"Directory does not exist: {experiment_folder}")

    # Ordenar los datos según el orden especificado
    import re

    def custom_sort_key(item):
        filename = item['filename']
        # Prioridad para archivos "00"
        if "00" in filename:
            return (0, filename)
        else:
            # Extraer número y tipo de archivo
            match = re.match(r"\((\d+)\)(mednegra|medroja|medamarilla|medmorada|medverde|medazul|mednaranja)\.txt", filename)
            if match:
                num = int(match.group(1))
                tipo = match.group(2)
                tipo_order = 0 if tipo == "mednegra" else 1  # mednegra antes que medcolor
                return (1, num, tipo_order)
            else:
                return (2, filename)

    # Ordenar los datos
    all_data_sorted = sorted(all_data, key=custom_sort_key)

    # Añadir los datos ordenados al Excel
    for data_item in all_data_sorted:
        add_data_to_excel(sheet, data_item['stats_formatted'], data_item['date'], data_item['session_name'], data_item['filename'], data_item['lab'])

    # Ruta de salida del archivo Excel
    excel_output_path = os.path.join(root_folder, f"General_Statistics_{date}.xlsx")

    # Asegurarse de que el directorio existe antes de guardar
    os.makedirs(root_folder, exist_ok=True)

    # Guardando el archivo Excel al final de todos los experimentos
    workbook.save(excel_output_path)
    print("Archivo Excel generado exitosamente en:", excel_output_path)

process_all_experiments(root_folder)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import datetime
import os
import glob

# Función para leer datos del sensor
def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                try:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append([float(num) for num in parts[1].strip('()').split(',')])
                except ValueError as e:
                    print(f"Error processing line in file {filepath}: {line}\n{e}")
    return timestamps, np.array(values)

# Función para graficar comparación entre dos laboratorios
def plot_comparison_labs(baseline_file_lab1, exp_file_lab1, color_lab1, baseline_file_lab2, exp_file_lab2, color_lab2, output_directory, shift, date_str, lab1_name, lab2_name):
    try:
        _, baseline_values_lab1 = read_data(baseline_file_lab1)
        _, exp_values_lab1 = read_data(exp_file_lab1)
        _, baseline_values_lab2 = read_data(baseline_file_lab2)
        _, exp_values_lab2 = read_data(exp_file_lab2)
    except Exception as e:
        print(f"Error reading data files:\n{e}")
        return

    shift_label = "Afternoon" if shift == "V" else "Morning"

    fig = plt.figure(figsize=(24, 10))

    # Subgráfica para el laboratorio 1
    ax_lab1 = fig.add_subplot(121, projection='3d')
    ax_lab1.scatter(baseline_values_lab1[:, 0], baseline_values_lab1[:, 1], baseline_values_lab1[:, 2], c='black', label=os.path.basename(baseline_file_lab1), s=50, alpha=0.6)
    ax_lab1.scatter(exp_values_lab1[:, 0], exp_values_lab1[:, 1], exp_values_lab1[:, 2], c=color_lab1, label=os.path.basename(exp_file_lab1), s=50, alpha=0.6)
    ax_lab1.set_title(f'{lab1_name} - {shift_label} Shift - Date: {date_str}', fontsize=20)
    ax_lab1.set_xlabel('VOLTS', fontsize=15)
    ax_lab1.set_ylabel('VOLTS', fontsize=15)
    ax_lab1.set_zlabel('VOLTS', fontsize=15)
    legend_lab1 = ax_lab1.legend(fontsize=14, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend_lab1.legend_handles:
        handle.set_sizes([300])

    # Subgráfica para el laboratorio 2
    ax_lab2 = fig.add_subplot(122, projection='3d')
    ax_lab2.scatter(baseline_values_lab2[:, 0], baseline_values_lab2[:, 1], baseline_values_lab2[:, 2], c='black', label=os.path.basename(baseline_file_lab2), s=50, alpha=0.6)
    ax_lab2.scatter(exp_values_lab2[:, 0], exp_values_lab2[:, 1], exp_values_lab2[:, 2], c=color_lab2, label=os.path.basename(exp_file_lab2), s=50, alpha=0.6)
    ax_lab2.set_title(f'{lab2_name} - {shift_label} Shift - Date: {date_str}', fontsize=20)
    ax_lab2.set_xlabel('VOLTS', fontsize=15)
    ax_lab2.set_ylabel('VOLTS', fontsize=15)
    ax_lab2.set_zlabel('VOLTS', fontsize=15)
    legend_lab2 = ax_lab2.legend(fontsize=14, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend_lab2.legend_handles:
        handle.set_sizes([300])

    # Generar imágenes en cada ángulo de 90°
    angles = [0, 90, 180, 270]
    for angle in angles:
        ax_lab1.view_init(elev=10, azim=angle)
        ax_lab2.view_init(elev=10, azim=angle)
        output_filename = f"{os.path.basename(baseline_file_lab1).replace('mednegra', f'{lab1_name}_vs_{lab2_name}_{angle}deg')}.png"
        output_filepath = os.path.join(output_directory, output_filename)
        plt.savefig(output_filepath, dpi=300)
        print(f"Image saved at {output_filepath}")

    plt.close(fig)

# Función principal para procesar los archivos de todos los laboratorios
def main(root_folder, labs):
    date_str = os.path.basename(root_folder)
    date_str = date_str.replace("-2", "").strip()
    shifts = ["V", "M"]

    color_map = {
        "medroja.txt": "red",
        "medmorada.txt": "purple",
        "medazul.txt": "blue",
        "medverde.txt": "green",
        "medamarilla.txt": "yellow",
        "mednaranja.txt":"orange"
    }

    # Combinaciones de laboratorios
    lab_combinations = [(labs[i], labs[j]) for i in range(len(labs)) for j in range(i + 1, len(labs))]

    for shift in shifts:
        for lab1, lab2 in lab_combinations:
            lab1_dirs = glob.glob(os.path.join(root_folder, f"*{lab1}"))
            lab2_dirs = glob.glob(os.path.join(root_folder, f"*{lab2}"))

            if not lab1_dirs or not lab2_dirs:
                print(f"Directorio no encontrado para {lab1} o {lab2}.")
                continue

            lab1_dir = lab1_dirs[0]
            lab2_dir = lab2_dirs[0]

            lab1_baseline_dir = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Curl_Baseline")
            lab1_exp_dir = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Curl_Experimental_Color")
            lab2_baseline_dir = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Curl_Baseline")
            lab2_exp_dir = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

            output_directory = os.path.join(root_folder, f"Data Analysis","3D_Curl_images", f"{lab1}_vs_{lab2}", shift)
            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            if not os.path.exists(lab1_baseline_dir) or not os.path.exists(lab1_exp_dir) or not os.path.exists(lab2_baseline_dir) or not os.path.exists(lab2_exp_dir):
                print(f"Faltan directorios para comparación entre {lab1} y {lab2}.")
                continue

            lab1_baseline_files = [f for f in os.listdir(lab1_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
            lab2_baseline_files = [f for f in os.listdir(lab2_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
            lab1_exp_files = [f for f in os.listdir(lab1_exp_dir) if any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]
            lab2_exp_files = [f for f in os.listdir(lab2_exp_dir) if any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]

            if not lab1_baseline_files or not lab2_baseline_files or not lab1_exp_files or not lab2_exp_files:
                print(f"No hay suficientes archivos para comparar {lab1} y {lab2}.")
                continue

            combinations_lab1 = generate_combinations(lab1_baseline_files, lab1_exp_files)
            combinations_lab2 = generate_combinations(lab2_baseline_files, lab2_exp_files)

            for (base_lab1, exp_lab1), (base_lab2, exp_lab2) in zip(combinations_lab1, combinations_lab2):
                baseline_file_lab1 = os.path.join(lab1_baseline_dir, base_lab1)
                experimental_file_lab1 = os.path.join(lab1_exp_dir, exp_lab1)
                baseline_file_lab2 = os.path.join(lab2_baseline_dir, base_lab2)
                experimental_file_lab2 = os.path.join(lab2_exp_dir, exp_lab2)

                if os.path.exists(baseline_file_lab1) and os.path.exists(experimental_file_lab1) and os.path.exists(baseline_file_lab2) and os.path.exists(experimental_file_lab2):
                    exp_suffix_lab1 = ''.join([i for i in os.path.basename(experimental_file_lab1).split('curl_')[-1] if not i.isdigit()])
                    exp_suffix_lab2 = ''.join([i for i in os.path.basename(experimental_file_lab2).split('curl_')[-1] if not i.isdigit()])
                    color_lab1 = color_map.get(exp_suffix_lab1, 'black')
                    color_lab2 = color_map.get(exp_suffix_lab2, 'black')
                    plot_comparison_labs(baseline_file_lab1, experimental_file_lab1, color_lab1, baseline_file_lab2, experimental_file_lab2, color_lab2, output_directory, shift, date_str, lab1, lab2)

# Generar combinaciones
def generate_combinations(baseline_files, exp_files):
    combinations = []
    for base_file in baseline_files:
        base_num = int(base_file.split('med')[0][-2:])
        for exp_file in exp_files:
            exp_num = int(exp_file.split('med')[0][-2:])
            if exp_num == base_num or exp_num == base_num - 1:
                combinations.append((base_file, exp_file))
    return combinations

if __name__ == "__main__":

    main(root_folder, labs)


## 2

$$

    \Huge \text{Boxplots}

$$

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import iqr
import re
from itertools import combinations

# Función para leer datos de sensores
def read_sensor_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
        return timestamps, np.array(data).T
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return None, None

# Función para calcular estadísticas de cada sensor
def calibrate_sensor(data, filename):
    try:
        results = {
            'mean': np.mean(data, axis=1),
            'median': np.median(data, axis=1),
            'variance': np.var(data, axis=1),
            'stdDev': np.std(data, axis=1),
            'min': np.min(data, axis=1),
            'max': np.max(data, axis=1),
            'firstQuartile': np.percentile(data, 25, axis=1),
            'thirdQuartile': np.percentile(data, 75, axis=1),
            'iqr': iqr(data, axis=1),
            'trimmedVariance': [np.var(np.trim_zeros(np.sort(sensor_data))) for sensor_data in data]
        }
        return results
    except Exception as e:
        print(f"Error al calcular estadísticas para el archivo {filename}: {e}")
        return None

# Función para buscar la carpeta del experimento
def find_experiment_folder(root_folder, lab, session):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and session in subfolder:
                    return os.path.join(root_folder, folder, subfolder)
    return None

# Función para generar boxplots individuales
def generate_boxplot(data, filename, output_folder, lab, session):
    if data is None or len(data) == 0:
        print(f"Advertencia: Datos no válidos para el archivo {filename}. Se omite la generación de boxplot.")
        return

    labels = ["Frontal", "Rear", "Left", "Right", "Top", "Bottom"]
    color_dict = {
        'negra': 'black',
        'roja': 'red',
        'morada': 'purple',
        'azul': 'blue',
        'verde': 'green',
        'amarilla': 'yellow',
        'naranja': 'orange'
    }
    file_base_name = filename.split('.')[0]
    color_suffix = file_base_name.split('med')[-1]
    color = color_dict.get(color_suffix, 'gray')  # Color por defecto si no se encuentra

    plt.figure(figsize=(10, 6))
    plt.boxplot(data.T, notch=True, patch_artist=True, boxprops=dict(facecolor=color))
    plt.title(f'Box Plot for {filename} - {lab} - {session}')
    plt.xlabel('Side of Cube')
    plt.ylabel('Volts')
    plt.xticks(range(1, len(labels) + 1), labels, rotation=45)
    plt.savefig(os.path.join(output_folder, f"{file_base_name}_{lab}_{session}_boxplot.png"))
    plt.grid(True)
    plt.close()

# Función para generar boxplots de comparación
def generate_comparison_boxplot(data_a, data_b, filename, output_folder, session, lab_a, lab_b, color_a, color_b):
    # Validar que data_a y data_b no sean None y tengan contenido
    if data_a is None and data_b is None:
        print(f"Advertencia: Datos no válidos para el archivo {filename}. No se genera comparación.")
        return

    labels = ["Frontal", "Rear", "Left", "Right", "Top", "Bottom"]

    # Si data_b no existe, generar un array ficticio con la media de data_a
    if data_a is not None and (data_b is None or data_b.size == 0):
        data_b = np.tile(np.mean(data_a, axis=1, keepdims=True), (1, data_a.shape[1]))

    # Si data_a no existe, generar un array ficticio con la media de data_b
    if data_b is not None and (data_a is None or data_a.size == 0):
        data_a = np.tile(np.mean(data_b, axis=1, keepdims=True), (1, data_b.shape[1]))

    combined_data = [data_a[i] for i in range(data_a.shape[0])] + [data_b[i] for i in range(data_b.shape[0])]
    positions = list(range(1, data_a.shape[0] + 1)) + list(range(data_a.shape[0] + 2, data_a.shape[0] + data_b.shape[0] + 2))
    colors = [color_a] * data_a.shape[0] + [color_b] * data_b.shape[0]

    plt.figure(figsize=(12, 8))
    boxprops = dict(facecolor='white')
    bplots = plt.boxplot(combined_data, positions=positions, notch=True, patch_artist=True, boxprops=boxprops)

    for patch, color in zip(bplots['boxes'], colors):
        patch.set_facecolor(color)

    plt.title(f'Comparison Box Plot for {filename} - {session}\n{lab_a} (left) vs {lab_b} (right)')
    plt.xlabel('Side of Cube')
    plt.ylabel('Volts')
    plt.xticks(range(1, len(labels) * 2 + 1), labels * 2, rotation=45)
    plt.legend(handles=[plt.Line2D([0], [0], color=color_a, lw=4, label=f'{lab_a} (Real)'),
                        plt.Line2D([0], [0], color=color_b, lw=4, label=f'{lab_b} (Ficticio)')],
               loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=2)
    plt.savefig(os.path.join(output_folder, f"{filename}_comparison_{session}_{lab_a}_vs_{lab_b}_boxplot.png"))
    plt.grid(True)
    plt.close()

# Función para analizar las carpetas y generar boxplots individuales
def analyze_folder(folder_path, output_folder, lab, session):
    file_stats = {}
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if any(filename.startswith("0") and filename.endswith(ext) for ext in ["mednegra.txt", "medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "naranja.txt"]):
                file_path = os.path.join(root, filename)
                timestamps, data = read_sensor_data(file_path)
                if data is not None:
                    stats = calibrate_sensor(data, filename)
                    if stats is not None:
                        file_stats[filename] = {'stats': stats, 'data': data}
                        generate_boxplot(data, filename, output_folder, lab, session)
    return file_stats

# Función para procesar todos los experimentos en pares de laboratorios o con un lab ficticio
def process_all_experiments(root_folder):
    sessions = ["V", "M"]
    all_data = {session: {lab: {} for lab in labs} for session in sessions}

    # Procesar carpetas para cada laboratorio y sesión
    for lab in labs:
        for session in sessions:
            experiment_folder = find_experiment_folder(root_folder, lab, session)
            if experiment_folder and os.path.exists(experiment_folder):
                output_folder_base = os.path.join(experiment_folder, "Data Analysis", "Graphics")
                function_title = "Boxplots"
                output_folder = os.path.join(output_folder_base, function_title)
                os.makedirs(output_folder, exist_ok=True)
                file_stats = analyze_folder(experiment_folder, output_folder, lab, session)
                all_data[session][lab] = file_stats

    color_dict = {
        'negra': 'black',
        'roja': 'red',
        'morada': 'purple',
        'azul': 'blue',
        'verde': 'green',
        'amarilla': 'yellow',
        'naranja': 'orange'
    }

    # Generar comparaciones para cada sesión
    for session in sessions:
        global_output_folder = os.path.join(root_folder, "Data Analysis", "Global Comparisons", session)
        os.makedirs(global_output_folder, exist_ok=True)
        if len(labs) == 1:  # Si solo hay un laboratorio
            lab_a = labs[0]
            lab_b = "Ficticio"
            for filename in all_data[session][lab_a]:
                data_a = all_data[session][lab_a].get(filename, {}).get('data', None)
                if data_a is not None:
                    # Crear datos ficticios basados en la media de data_a
                    data_b = np.tile(np.mean(data_a, axis=1, keepdims=True), (1, data_a.shape[1]))
                    color_suffix_a = filename.split('med')[-1].split('.')[0] if filename in all_data[session][lab_a] else ''
                    color_a = color_dict.get(color_suffix_a, 'blue')
                    color_b = 'gray'  # Color para el laboratorio ficticio

                    generate_comparison_boxplot(data_a, data_b, filename, global_output_folder, session, lab_a, lab_b, color_a, color_b)
        else:  # Si hay más de un laboratorio
            for lab_a, lab_b in combinations(labs, 2):  # Combinaciones de laboratorios de 2 en 2
                all_filenames = set(all_data[session][lab_a].keys()) | set(all_data[session][lab_b].keys())  # Unión de archivos de ambos laboratorios
                for filename in all_filenames:
                    # Datos del primer laboratorio (lab_a)
                    data_a = all_data[session][lab_a].get(filename, {}).get('data', None)

                    # Datos del segundo laboratorio (lab_b)
                    data_b = all_data[session][lab_b].get(filename, {}).get('data', None)

                    # Obtener colores para cada laboratorio
                    color_suffix_a = filename.split('med')[-1].split('.')[0] if filename in all_data[session][lab_a] else ''
                    color_a = color_dict.get(color_suffix_a, 'blue')

                    color_suffix_b = filename.split('med')[-1].split('.')[0] if filename in all_data[session][lab_b] else ''
                    color_b = color_dict.get(color_suffix_b, 'green')

                    # Generar boxplot comparativo, incluso si faltan datos en uno de los laboratorios
                    generate_comparison_boxplot(data_a, data_b, filename, global_output_folder, session, lab_a, lab_b, color_a, color_b)

# Procesar todos los experimentos
process_all_experiments(root_folder)


## 3

$$

    \Huge \text{Primera Integral Helicidad}

$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from itertools import combinations

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                values.append(float(parts[1]))
    return np.array(values)

def get_color(filename):
    color_map = {
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow',
        'mednaranja.txt': 'orange'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

def get_experimental_info(filename):
    """Extrae el número, color y código de triada del archivo experimental."""
    number = extract_number(filename)
    color_match = re.search(r'med(\w+)\.txt', filename)
    triad_code_match = re.search(r'helicity_(\w+)_', filename)
    color = color_match.group(1) if color_match else 'Unknown'
    triad_code = triad_code_match.group(1).replace("_velocity", "") if triad_code_match else 'Unknown'  # Elimina "velocity"
    return number, color, triad_code

def get_triad_name(triad_code):
    return triad_names.get(triad_code, "Código de triada no encontrado")

def plot_comparison(base_file1, base_file2, experimental_file, output_dir, shift, date_str, lab_name):
    number, color, triad_code = get_experimental_info(experimental_file)
    base_values1 = read_data(base_file1)
    base_values2 = read_data(base_file2)
    experimental_values = read_data(experimental_file)
    triad_name = get_triad_name(triad_code)

    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(np.arange(len(base_values1)), base_values1, c='black', label=os.path.basename(base_file1))
    ax.plot(np.arange(len(base_values2)), base_values2, c='gray', label=os.path.basename(base_file2))
    ax.plot(np.arange(len(experimental_values)), experimental_values, c=get_color(experimental_file), label=os.path.basename(experimental_file))

    ax.set_title(f'{lab_name} - {shift} - {triad_name} - Date: {date_str}', fontsize=14)
    ax.set_xlabel('Record Number')
    ax.set_ylabel('V^2*m^2')
    ax.grid(True)

    max_length = max(len(base_values1), len(base_values2), len(experimental_values))
    ax.set_xlim(0, max_length)

    ax.legend()

    plt.tight_layout()

    number, color, triad_code = get_experimental_info(experimental_file)
    output_filename = f"SAH_LB_{number}_&_{int(number)+1}_VS_MEI_{color.upper()}_{number}_{triad_code}.png"

    # Crear subcarpeta para la triada
    triad_folder = os.path.join(output_dir, triad_code)
    if not os.path.exists(triad_folder):
        os.makedirs(triad_folder)

    output_filepath = os.path.join(triad_folder, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    ax.grid(True)
    plt.close(fig)

def plot_comparison_two_labs(base_file1_lab1, base_file2_lab1, exp_file_lab1,
                             base_file1_lab2, base_file2_lab2, exp_file_lab2,
                             output_dir, shift, date_str, lab1, lab2):
    base_values1_lab1 = read_data(base_file1_lab1)
    base_values2_lab1 = read_data(base_file2_lab1)
    experimental_values_lab1 = read_data(exp_file_lab1)
    
    base_values1_lab2 = read_data(base_file1_lab2)
    base_values2_lab2 = read_data(base_file2_lab2)
    experimental_values_lab2 = read_data(exp_file_lab2)
    number, color, triad_code = get_experimental_info(exp_file_lab1)

    triad_name = get_triad_name(triad_code)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    ax1.plot(np.arange(len(base_values1_lab1)), base_values1_lab1, c='black', label=os.path.basename(base_file1_lab1))
    ax1.plot(np.arange(len(base_values2_lab1)), base_values2_lab1, c='gray', label=os.path.basename(base_file2_lab1))
    ax1.plot(np.arange(len(experimental_values_lab1)), experimental_values_lab1, c=get_color(exp_file_lab1), label=os.path.basename(exp_file_lab1))
    ax1.set_title(f'{lab1} - {shift} - {triad_name} - Date: {date_str}')
    ax1.set_xlabel('Record Number')
    ax1.set_ylabel('V^2*m^2')
    ax1.legend()
    ax1.grid(True)

    ax2.plot(np.arange(len(base_values1_lab2)), base_values1_lab2, c='black', label=os.path.basename(base_file1_lab2))
    ax2.plot(np.arange(len(base_values2_lab2)), base_values2_lab2, c='gray', label=os.path.basename(base_file2_lab2))
    ax2.plot(np.arange(len(experimental_values_lab2)), experimental_values_lab2, c=get_color(exp_file_lab2), label=os.path.basename(exp_file_lab2))
    ax2.set_title(f'{lab2} - {shift} - {triad_name} - Date: {date_str}')
    ax2.set_xlabel('Record Number')
    ax2.set_ylabel('V^2*m^2')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    
    number, color, triad_code = get_experimental_info(exp_file_lab1)
    output_filename = f"SAH_LB_{number}_&_{int(number)+1}_VS_MEI_{color.upper()}_{number}_{triad_code}_{lab1}_vs_{lab2}.png"

    # Crear subcarpeta para la triada
    triad_folder = os.path.join(output_dir, triad_code)
    if not os.path.exists(triad_folder):
        os.makedirs(triad_folder)

    output_filepath = os.path.join(triad_folder, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved combined plot as {output_filepath}")
    plt.close(fig)

def get_triad_code(filename):
    """Extrae el código de triada del archivo experimental."""
    triad_code_match = re.search(r'helicity_(\w+)_', filename)
    triad_code = triad_code_match.group(1).replace("_velocity", "") if triad_code_match else 'Unknown'  # Elimina "velocity"
    return triad_code

def generate_combinations(baseline_files, experimental_files):
    """Genera combinaciones de archivos base y experimentales que coinciden por triada y número."""
    combinations_list = []
    baseline_files_sorted = sorted(baseline_files, key=lambda x: int(extract_number(x)))
    
    for i in range(len(baseline_files_sorted) - 1):
        base_file1 = baseline_files_sorted[i]
        base_file2 = baseline_files_sorted[i + 1]
        
        triad_code1 = get_triad_code(base_file1)
        triad_code2 = get_triad_code(base_file2)

        num1 = extract_number(base_file1)
        num2 = extract_number(base_file2)
        
        if num1 and num2 and int(num2) == int(num1) + 1 and triad_code1 == triad_code2:
            for exp_file in experimental_files:
                exp_num = extract_number(exp_file)
                exp_triad = get_triad_code(exp_file)
                if exp_num == num1 and triad_code1 == exp_triad:
                    combinations_list.append((base_file1, base_file2, exp_file))
    
    return combinations_list

def main():
    shifts = ["V", "M"]

    if len(labs) == 1:
        lab1 = labs[0]
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab1}"))
            if not lab_dirs:
                print(f"Omitiendo cálculo para {lab1} en turno {shift} ya que el directorio no existe.")
                continue

            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Helicity_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Helicity_Experimental_Color")

            output_directory = os.path.join(root_folder, "Data Analysis", "Graphics", "Sum_Helicity", f"{lab1}", shift)

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab1} en turno {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            for triad_code in triad_codes.values():
                baseline_files = []
                experimental_files = []
                baseline_files.extend(glob.glob(os.path.join(baseline_dir, triad_code, 'sum_helicity_*mednegra.txt')))
                experimental_files.extend(glob.glob(os.path.join(exp_dir, triad_code, 'sum_helicity_*med*.txt')))

                all_combinations = generate_combinations(baseline_files, experimental_files)

                for base_file1, base_file2, exp_file in all_combinations:
                    print(f"Procesando: {base_file1}, {base_file2}, {exp_file}")
                    plot_comparison(base_file1, base_file2, exp_file, output_directory, shift, date_str, f"{lab1}")

    else:
        lab_pairs = list(combinations(labs, 2))
        processed_combinations = set()

        for lab1, lab2 in lab_pairs:
            for shift in shifts:
                lab1_dirs = glob.glob(os.path.join(root_folder, f"*{lab1}"))
                lab2_dirs = glob.glob(os.path.join(root_folder, f"*{lab2}"))

                if not lab1_dirs or not lab2_dirs:
                    print(f"Omitiendo cálculo para {lab1} y {lab2} en turno {shift} ya que el directorio no existe.")
                    continue

                lab1_dir = lab1_dirs[0]
                lab2_dir = lab2_dirs[0]
                date_str = os.path.basename(lab1_dir).split(' ')[0]
                print(lab1_dir, lab2_dir)

                baseline_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Helicity_Baseline")
                exp_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Helicity_Experimental_Color")

                baseline_dir2 = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Sum_Helicity_Baseline")
                exp_dir2 = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Sum_Helicity_Experimental_Color")

                output_directory = os.path.join(root_folder, "Data Analysis", "Graphics", "Sum_Helicity", f"{lab1} vs {lab2}", shift)

                if not os.path.exists(output_directory):
                    os.makedirs(output_directory)

                for triad_code in triad_codes.values():
                    baseline_files1 = []
                    experimental_files1 = []
                    baseline_files2 = []
                    experimental_files2 = []
                    baseline_files1.extend(glob.glob(os.path.join(baseline_dir1, triad_code, 'sum_helicity_*mednegra.txt')))
                    experimental_files1.extend(glob.glob(os.path.join(exp_dir1, triad_code, 'sum_helicity_*med*.txt')))
                    baseline_files2.extend(glob.glob(os.path.join(baseline_dir2, triad_code, 'sum_helicity_*mednegra.txt')))
                    experimental_files2.extend(glob.glob(os.path.join(exp_dir2, triad_code, 'sum_helicity_*med*.txt'))) 

                    all_combinations1 = generate_combinations(baseline_files1, experimental_files1)
                    all_combinations2 = generate_combinations(baseline_files2, experimental_files2)

                    for (base_file1_lab1, base_file2_lab1, exp_file_lab1), (base_file1_lab2, base_file2_lab2, exp_file_lab2) in zip(all_combinations1, all_combinations2):
                        if extract_number(exp_file_lab1) == extract_number(exp_file_lab2):
                            combination_key = (os.path.basename(base_file1_lab1), os.path.basename(base_file2_lab1), os.path.basename(exp_file_lab1),
                                           os.path.basename(base_file1_lab2), os.path.basename(base_file2_lab2), os.path.basename(exp_file_lab2))

                            if combination_key not in processed_combinations:
                                print(f"Procesando combinacion de archivos: {combination_key}")
                                plot_comparison_two_labs(base_file1_lab1, base_file2_lab1, exp_file_lab1,
                                                     base_file1_lab2, base_file2_lab2, exp_file_lab2,
                                                     output_directory, shift, date_str, lab1, lab2)
                                processed_combinations.add(combination_key)
                            else:
                                print(f"Combinación repetida, saltando: {combination_key}")

if __name__ == "__main__":
    main()

## 4

$$

    \Huge \text{Primera Integral Helicidad Absoluta}

$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from itertools import combinations

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                values.append(float(parts[1]))
    return np.array(values)

def get_color(filename):
    color_map = {
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow',
        'mednaranja.txt': 'orange'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

def get_experimental_info(filename):
    """Extrae el número, color y código de triada del archivo experimental."""
    number = extract_number(filename)
    color_match = re.search(r'med(\w+)\.txt', filename)
    triad_code_match = re.search(r'helicity_(\w+)_', filename)
    color = color_match.group(1) if color_match else 'Unknown'
    triad_code = triad_code_match.group(1).replace("_velocity", "") if triad_code_match else 'Unknown'  # Elimina "velocity"
    return number, color, triad_code

def get_triad_name(triad_code):
    return triad_names.get(triad_code, "Código de triada no encontrado")

def plot_comparison(base_file1, base_file2, experimental_file, output_dir, shift, date_str, lab_name):
    number, color, triad_code = get_experimental_info(experimental_file)
    base_values1 = read_data(base_file1)
    base_values2 = read_data(base_file2)
    experimental_values = read_data(experimental_file)
    triad_name = get_triad_name(triad_code)

    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(np.arange(len(base_values1)), base_values1, c='black', label=os.path.basename(base_file1))
    ax.plot(np.arange(len(base_values2)), base_values2, c='gray', label=os.path.basename(base_file2))
    ax.plot(np.arange(len(experimental_values)), experimental_values, c=get_color(experimental_file), label=os.path.basename(experimental_file))

    ax.set_title(f'{lab_name} - {shift} - {triad_name} - Date: {date_str}', fontsize=14)
    ax.set_xlabel('Record Number')
    ax.set_ylabel('V^2*m^2')
    ax.grid(True)

    max_length = max(len(base_values1), len(base_values2), len(experimental_values))
    ax.set_xlim(0, max_length)

    ax.legend()

    plt.tight_layout()

    number, color, triad_code = get_experimental_info(experimental_file)
    output_filename = f"SAVAH_LB_{number}_&_{int(number)+1}_VS_MEI_{color.upper()}_{number}_{triad_code}.png"

    # Crear subcarpeta para la triada
    triad_folder = os.path.join(output_dir, triad_code)
    if not os.path.exists(triad_folder):
        os.makedirs(triad_folder)

    output_filepath = os.path.join(triad_folder, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def plot_comparison_two_labs(base_file1_lab1, base_file2_lab1, exp_file_lab1,
                             base_file1_lab2, base_file2_lab2, exp_file_lab2,
                             output_dir, shift, date_str, lab1, lab2):
    base_values1_lab1 = read_data(base_file1_lab1)
    base_values2_lab1 = read_data(base_file2_lab1)
    experimental_values_lab1 = read_data(exp_file_lab1)
    
    base_values1_lab2 = read_data(base_file1_lab2)
    base_values2_lab2 = read_data(base_file2_lab2)
    experimental_values_lab2 = read_data(exp_file_lab2)
    number, color, triad_code = get_experimental_info(exp_file_lab1)
    triad_name = get_triad_name(triad_code)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    ax1.plot(np.arange(len(base_values1_lab1)), base_values1_lab1, c='black', label=os.path.basename(base_file1_lab1))
    ax1.plot(np.arange(len(base_values2_lab1)), base_values2_lab1, c='gray', label=os.path.basename(base_file2_lab1))
    ax1.plot(np.arange(len(experimental_values_lab1)), experimental_values_lab1, c=get_color(exp_file_lab1), label=os.path.basename(exp_file_lab1))
    ax1.set_title(f'{lab1} - {shift} - {triad_name} - Date: {date_str}')
    ax1.set_xlabel('Record Number')
    ax1.set_ylabel('V^2*m^2')
    ax1.legend()
    ax1.grid(True)

    ax2.plot(np.arange(len(base_values1_lab2)), base_values1_lab2, c='black', label=os.path.basename(base_file1_lab2))
    ax2.plot(np.arange(len(base_values2_lab2)), base_values2_lab2, c='gray', label=os.path.basename(base_file2_lab2))
    ax2.plot(np.arange(len(experimental_values_lab2)), experimental_values_lab2, c=get_color(exp_file_lab2), label=os.path.basename(exp_file_lab2))
    ax2.set_title(f'{lab2} - {shift} - {triad_name} - Date: {date_str}')
    ax2.set_xlabel('Record Number')
    ax2.set_ylabel('V^2*m^2')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    
    number, color, triad_code = get_experimental_info(exp_file_lab1)
    output_filename = f"SAVAH_LB_{number}_&_{int(number)+1}_VS_MEI_{color.upper()}_{number}_{triad_code}_{lab1}_vs_{lab2}.png"

    # Crear subcarpeta para la triada
    triad_folder = os.path.join(output_dir, triad_code)
    if not os.path.exists(triad_folder):
        os.makedirs(triad_folder)

    output_filepath = os.path.join(triad_folder, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved combined plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files):
    """Genera combinaciones de archivos base y experimentales que coinciden por triada y número."""
    combinations_list = []
    baseline_files_sorted = sorted(baseline_files, key=lambda x: int(extract_number(x)))
    
    for i in range(len(baseline_files_sorted) - 1):
        base_file1 = baseline_files_sorted[i]
        base_file2 = baseline_files_sorted[i + 1]
        
        triad_code1 = get_triad_code(base_file1)
        triad_code2 = get_triad_code(base_file2)

        num1 = extract_number(base_file1)
        num2 = extract_number(base_file2)
        
        if num1 and num2 and int(num2) == int(num1) + 1 and triad_code1 == triad_code2:
            for exp_file in experimental_files:
                exp_num = extract_number(exp_file)
                exp_triad = get_triad_code(exp_file)
                if exp_num == num1 and triad_code1 == exp_triad:
                    combinations_list.append((base_file1, base_file2, exp_file))
    
    return combinations_list

def main():
    shifts = ["V", "M"]

    if len(labs) == 1:
        lab1 = labs[0]
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab1}"))
            if not lab_dirs:
                print(f"Omitiendo cálculo para {lab1} en turno {shift} ya que el directorio no existe.")
                continue

            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Experimental_Color")

            output_directory = os.path.join(root_folder, "Data Analysis", "Graphics", "Sum_Absolute_Helicity", f"{lab1}", shift)

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab1} en turno {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            for triad_code in triad_codes.values():
                baseline_files = []
                experimental_files = []
                baseline_files.extend(glob.glob(os.path.join(baseline_dir, triad_code, 'sum_absolute_helicity_*mednegra.txt')))
                experimental_files.extend(glob.glob(os.path.join(exp_dir, triad_code, 'sum_absolute_helicity_*med*.txt')))

                all_combinations = generate_combinations(baseline_files, experimental_files)

                for base_file1, base_file2, exp_file in all_combinations:
                    plot_comparison(base_file1, base_file2, exp_file, output_directory, shift, date_str, f"{lab1}")

    else:
        lab_pairs = list(combinations(labs, 2))
        processed_combinations = set()

        for lab1, lab2 in lab_pairs:
            for shift in shifts:
                lab1_dirs = glob.glob(os.path.join(root_folder, f"*{lab1}"))
                lab2_dirs = glob.glob(os.path.join(root_folder, f"*{lab2}"))

                if not lab1_dirs or not lab2_dirs:
                    print(f"Omitiendo cálculo para {lab1} y {lab2} en turno {shift} ya que el directorio no existe.")
                    continue

                lab1_dir = lab1_dirs[0]
                lab2_dir = lab2_dirs[0]
                date_str = os.path.basename(lab1_dir).split(' ')[0]

                baseline_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Baseline")
                exp_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Experimental_Color")

                baseline_dir2 = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Baseline")
                exp_dir2 = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Experimental_Color")

                output_directory = os.path.join(root_folder, "Data Analysis", "Graphics", "Sum_Absolute_Helicity", f"{lab1} vs {lab2}", shift)

                if not os.path.exists(output_directory):
                    os.makedirs(output_directory)

                for triad_code in triad_codes.values():
                    baseline_files1 = []
                    experimental_files1 = []
                    baseline_files2 = []
                    experimental_files2 = []
                    baseline_files1.extend(glob.glob(os.path.join(baseline_dir1, triad_code, 'sum_absolute_helicity_*mednegra.txt')))
                    experimental_files1.extend(glob.glob(os.path.join(exp_dir1, triad_code, 'sum_absolute_helicity_*med*.txt')))
                    baseline_files2.extend(glob.glob(os.path.join(baseline_dir2, triad_code, 'sum_absolute_helicity_*mednegra.txt')))
                    experimental_files2.extend(glob.glob(os.path.join(exp_dir2, triad_code, 'sum_absolute_helicity_*med*.txt')))

                    all_combinations1 = generate_combinations(baseline_files1, experimental_files1)
                    all_combinations2 = generate_combinations(baseline_files2, experimental_files2)

                    for (base_file1_lab1, base_file2_lab1, exp_file_lab1), (base_file1_lab2, base_file2_lab2, exp_file_lab2) in zip(all_combinations1, all_combinations2):

                        if extract_number(exp_file_lab1) == extract_number(exp_file_lab2):
                            combination_key = (os.path.basename(base_file1_lab1), os.path.basename(base_file2_lab1), os.path.basename(exp_file_lab1),
                                            os.path.basename(base_file1_lab2), os.path.basename(base_file2_lab2), os.path.basename(exp_file_lab2))

                            if combination_key not in processed_combinations:
                                plot_comparison_two_labs(base_file1_lab1, base_file2_lab1, exp_file_lab1,
                                                     base_file1_lab2, base_file2_lab2, exp_file_lab2,
                                                     output_directory, shift, date_str, lab1, lab2)
                                processed_combinations.add(combination_key)

if __name__ == "__main__":
    main()

## 5

$$

    \Huge \text{Primera Integral Enstrofía}

$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from itertools import combinations

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                values.append(float(parts[1]))
    return np.array(values)

def get_color(filename):
    color_map = {
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow',
        'mednaranja.txt': 'orange'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'

def get_experimental_info(filename):
    """Extrae el número, color y código de triada del archivo experimental."""
    number = extract_number(filename)
    color_match = re.search(r'med(\w+)\.txt', filename)
    triad_code_match = re.search(r'enstrophy_vorticity_(\w+)_velocity', filename)
    color = color_match.group(1) if color_match else 'Unknown'
    triad_code = triad_code_match.group(1) if triad_code_match else 'Unknown'
    return number, color, triad_code

def get_triad_name(triad_code):
    return triad_names.get(triad_code, "Código de triada no encontrado")

def plot_comparison(base_file1, base_file2, experimental_file, output_dir, shift, date_str, lab_name):
    base_values1 = read_data(base_file1)
    base_values2 = read_data(base_file2)
    experimental_values = read_data(experimental_file)
    number, color, triad_code = get_experimental_info(experimental_file)
    triad_name = get_triad_name(triad_code)

    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(np.arange(len(base_values1)), base_values1, c='black', label=os.path.basename(base_file1))
    ax.plot(np.arange(len(base_values2)), base_values2, c='gray', label=os.path.basename(base_file2))
    ax.plot(np.arange(len(experimental_values)), experimental_values, c=get_color(experimental_file), label=os.path.basename(experimental_file))

    ax.set_title(f'{lab_name} - {shift} - {triad_name} - Date: {date_str}', fontsize=14)
    ax.set_xlabel('Record Number')
    ax.set_ylabel('V^2*m^2')
    ax.grid(True)

    max_length = max(len(base_values1), len(base_values2), len(experimental_values))
    ax.set_xlim(0, max_length)

    ax.legend()
    plt.tight_layout()

    number, color, triad_code = get_experimental_info(experimental_file)
    output_filename = f"SAE_LB_{number}_&_{int(number)+1}_VS_MEI_{color.upper()}_{number}_{triad_code}.png"

    triad_folder = os.path.join(output_dir, triad_code)
    if not os.path.exists(triad_folder):
        os.makedirs(triad_folder)

    output_filepath = os.path.join(triad_folder, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def plot_comparison_two_labs(base_file1_lab1, base_file2_lab1, exp_file_lab1,
                             base_file1_lab2, base_file2_lab2, exp_file_lab2,
                             output_dir, shift, date_str, lab1, lab2):
    base_values1_lab1 = read_data(base_file1_lab1)
    base_values2_lab1 = read_data(base_file2_lab1)
    experimental_values_lab1 = read_data(exp_file_lab1)
    
    base_values1_lab2 = read_data(base_file1_lab2)
    base_values2_lab2 = read_data(base_file2_lab2)
    experimental_values_lab2 = read_data(exp_file_lab2)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    number, color, triad_code = get_experimental_info(exp_file_lab1)
    triad_name = get_triad_name(triad_code)

    ax1.plot(np.arange(len(base_values1_lab1)), base_values1_lab1, c='black', label=os.path.basename(base_file1_lab1))
    ax1.plot(np.arange(len(base_values2_lab1)), base_values2_lab1, c='gray', label=os.path.basename(base_file2_lab1))
    ax1.plot(np.arange(len(experimental_values_lab1)), experimental_values_lab1, c=get_color(exp_file_lab1), label=os.path.basename(exp_file_lab1))
    ax1.set_title(f'{lab1} - {shift} - {triad_name} - Date: {date_str}')
    ax1.set_xlabel('Record Number')
    ax1.set_ylabel('V^2*m^2')
    ax1.legend()
    ax1.grid(True)

    ax2.plot(np.arange(len(base_values1_lab2)), base_values1_lab2, c='black', label=os.path.basename(base_file1_lab2))
    ax2.plot(np.arange(len(base_values2_lab2)), base_values2_lab2, c='gray', label=os.path.basename(base_file2_lab2))
    ax2.plot(np.arange(len(experimental_values_lab2)), experimental_values_lab2, c=get_color(exp_file_lab2), label=os.path.basename(exp_file_lab2))
    ax2.set_title(f'{lab2} - {shift} - {triad_name} - Date: {date_str}')
    ax2.set_xlabel('Record Number')
    ax2.set_ylabel('V^2*m^2')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    
    number, color, triad_code = get_experimental_info(exp_file_lab1)
    output_filename = f"SAE_LB_{number}_&_{int(number)+1}_VS_MEI_{color.upper()}_{number}_{triad_code}_{lab1}_vs_{lab2}.png"

    triad_folder = os.path.join(output_dir, triad_code)
    if not os.path.exists(triad_folder):
        os.makedirs(triad_folder)

    output_filepath = os.path.join(triad_folder, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved combined plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files):
    """Genera combinaciones de archivos base y experimentales que coinciden por triada y número."""
    combinations_list = []
    baseline_files_sorted = sorted(baseline_files, key=lambda x: int(extract_number(x)))
    
    for i in range(len(baseline_files_sorted) - 1):
        base_file1 = baseline_files_sorted[i]
        base_file2 = baseline_files_sorted[i + 1]
        
        triad_code1 = get_triad_code(base_file1)
        triad_code2 = get_triad_code(base_file2)

        num1 = extract_number(base_file1)
        num2 = extract_number(base_file2)
        
        if num1 and num2 and int(num2) == int(num1) + 1 and triad_code1 == triad_code2:
            for exp_file in experimental_files:
                exp_num = extract_number(exp_file)
                exp_triad = get_triad_code(exp_file)
                if exp_num == num1 and triad_code1 == exp_triad:
                    combinations_list.append((base_file1, base_file2, exp_file))
    
    return combinations_list

def main():
    shifts = ["V", "M"]

    if len(labs) == 1:
        lab1 = labs[0]
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab1}"))
            if not lab_dirs:
                print(f"Omitiendo cálculo para {lab1} en turno {shift} ya que el directorio no existe.")
                continue

            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Experimental_Color")

            output_directory = os.path.join(root_folder, "Data Analysis", "Graphics", "Sum_Enstrophy", f"{lab1}", shift)

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab1} en turno {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            for triad_code in triad_codes.values():
                baseline_files = []
                experimental_files = []
                baseline_files.extend(glob.glob(os.path.join(baseline_dir, triad_code, 'sum_enstrophy_vorticity_*mednegra.txt')))
                experimental_files.extend(glob.glob(os.path.join(exp_dir, triad_code, 'sum_enstrophy_vorticity_*med*.txt')))

                all_combinations = generate_combinations(baseline_files, experimental_files)

                for base_file1, base_file2, exp_file in all_combinations:
                    print(f"Procesando: {base_file1}, {base_file2}, {exp_file}")
                    plot_comparison(base_file1, base_file2, exp_file, output_directory, shift, date_str, f"{lab1}")

    else:
        lab_pairs = list(combinations(labs, 2))
        processed_combinations = set()

        for lab1, lab2 in lab_pairs:
            for shift in shifts:
                lab1_dirs = glob.glob(os.path.join(root_folder, f"*{lab1}"))
                lab2_dirs = glob.glob(os.path.join(root_folder, f"*{lab2}"))

                if not lab1_dirs or not lab2_dirs:
                    print(f"Omitiendo cálculo para {lab1} y {lab2} en turno {shift} ya que el directorio no existe.")
                    continue

                lab1_dir = lab1_dirs[0]
                lab2_dir = lab2_dirs[0]
                date_str = os.path.basename(lab1_dir).split(' ')[0]

                baseline_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Baseline")
                exp_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Experimental_Color")

                baseline_dir2 = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Baseline")
                exp_dir2 = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Experimental_Color")

                output_directory = os.path.join(root_folder, "Data Analysis", "Graphics", "Sum_Enstrophy", f"{lab1} vs {lab2}", shift)

                if not os.path.exists(output_directory):
                    os.makedirs(output_directory)

                for triad_code in triad_codes.values():
                    baseline_files1 = []
                    experimental_files1 = []
                    baseline_files2 = []
                    experimental_files2 = []
                    baseline_files1.extend(glob.glob(os.path.join(baseline_dir1, triad_code, 'sum_enstrophy_vorticity_*mednegra.txt')))
                    experimental_files1.extend(glob.glob(os.path.join(exp_dir1, triad_code, 'sum_enstrophy_vorticity_*med*.txt')))
                    baseline_files2.extend(glob.glob(os.path.join(baseline_dir2, triad_code, 'sum_enstrophy_vorticity_*mednegra.txt')))
                    experimental_files2.extend(glob.glob(os.path.join(exp_dir2, triad_code, 'sum_enstrophy_vorticity_*med*.txt')))

                    all_combinations1 = generate_combinations(baseline_files1, experimental_files1)
                    all_combinations2 = generate_combinations(baseline_files2, experimental_files2)

                    for (base_file1_lab1, base_file2_lab1, exp_file_lab1), (base_file1_lab2, base_file2_lab2, exp_file_lab2) in zip(all_combinations1, all_combinations2):
                        if extract_number(exp_file_lab1) == extract_number(exp_file_lab2):
                            combination_key = (os.path.basename(base_file1_lab1), os.path.basename(base_file2_lab1), os.path.basename(exp_file_lab1),
                                           os.path.basename(base_file1_lab2), os.path.basename(base_file2_lab2), os.path.basename(exp_file_lab2))

                            if combination_key not in processed_combinations:
                                print(f"Procesando combinacion de archivos: {combination_key}")
                                plot_comparison_two_labs(base_file1_lab1, base_file2_lab1, exp_file_lab1,
                                                     base_file1_lab2, base_file2_lab2, exp_file_lab2,
                                                     output_directory, shift, date_str, lab1, lab2)
                                processed_combinations.add(combination_key)
                            else:
                                print(f"Combinación repetida, saltando: {combination_key}")

if __name__ == "__main__":
    main()

## 7

$$

    \Huge \text{Modulos del Rotacional}

$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

def read_data(filepath):
    """Lee datos numéricos de un archivo y devuelve un array de valores."""
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                values.append(float(parts[1]))
    return np.array(values)

def get_color(filename):
    """Obtiene el color correspondiente según el sufijo del nombre de archivo."""
    color_map = {
        'mednegra.txt': 'black',
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow',
        'mednaranja.txt': 'orange'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

def plot_individual(lab_name, baseline_file, experimental_file, baseline_file2, output_dir, shift, date_str):
    """Genera gráficas individuales para un solo laboratorio."""
    # Leer los datos de los archivos
    baseline_values = read_data(baseline_file)
    experimental_values = read_data(experimental_file)
    baseline_values2 = read_data(baseline_file2)

    plt.figure(figsize=(10, 6))
    plt.plot(baseline_values, c='gray', label=os.path.basename(baseline_file))
    plt.plot(experimental_values, c=get_color(experimental_file), label=os.path.basename(experimental_file))
    plt.plot(baseline_values2, c='black', label=os.path.basename(baseline_file2))
    plt.title(f'{lab_name} - Shift: {"Vespertino" if shift == "V" else "Matutino"} - Date: {date_str}', fontsize=14)
    plt.xlabel('Register')
    plt.ylabel('Magnitude (V/m)')
    plt.grid(True)
    plt.legend()
    
    # Guardar la gráfica individual
    output_filename = f"Individual_Triad_{lab_name}_{os.path.basename(baseline_file).split('.')[0]}.png"
    output_filepath = os.path.join(output_dir, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved individual plot for {lab_name} as {output_filepath}")
    plt.close()

def plot_triad_comparison(baseline_file1, experimental_file1, baseline_file1_2,
                          baseline_file2, experimental_file2, baseline_file2_2,
                          lab1_name, lab2_name, output_dir, shift, date_str):
    """Genera una comparación de archivos de dos laboratorios en una gráfica lado a lado."""
    # Leer datos de los archivos de ambos laboratorios
    baseline_values1 = read_data(baseline_file1)
    experimental_values1 = read_data(experimental_file1)
    baseline_values1_2 = read_data(baseline_file1_2)
    
    baseline_values2 = read_data(baseline_file2)
    experimental_values2 = read_data(experimental_file2)
    baseline_values2_2 = read_data(baseline_file2_2)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))

    # Graficar Lab 1 con tres series de datos
    ax1.plot(baseline_values1, c='gray', label=os.path.basename(baseline_file1))
    ax1.plot(experimental_values1, c=get_color(experimental_file1), label=os.path.basename(experimental_file1))
    ax1.plot(baseline_values1_2, c='black', label=os.path.basename(baseline_file1_2))
    ax1.set_title(f'{lab1_name} - Shift: {"Vespertino" if shift == "V" else "Matutino"} - Date: {date_str}', fontsize=14)
    ax1.set_xlabel('Register')
    ax1.set_ylabel('Magnitude (V/m)')
    ax1.grid(True)
    ax1.legend()

    # Graficar Lab 2 con tres series de datos
    ax2.plot(baseline_values2, c='gray', label=os.path.basename(baseline_file2))
    ax2.plot(experimental_values2, c=get_color(experimental_file2), label=os.path.basename(experimental_file2))
    ax2.plot(baseline_values2_2, c='black', label=os.path.basename(baseline_file2_2))
    ax2.set_title(f'{lab2_name} - Shift: {"Vespertino" if shift == "V" else "Matutino"} - Date: {date_str}', fontsize=14)
    ax2.set_xlabel('Register')
    ax2.set_ylabel('Magnitude (V/m)')
    ax2.grid(True)
    ax2.legend()
    
    plt.tight_layout()

    # Generar un nombre único para cada gráfico
    output_filename = f"Comparison_Triad_{lab1_name}_vs_{lab2_name}_{os.path.basename(baseline_file1).split('.')[0]}_VS_{os.path.basename(baseline_file2).split('.')[0]}.png"
    output_filepath = os.path.join(output_dir, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.grid(True)
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files, baseline_files2):
    """Genera combinaciones de archivos según la estructura n y n+1."""
    combinations = []
    for base_file in baseline_files:
        base_num = int(base_file.split('med')[0][-2:])
        for exp_file in experimental_files:
            exp_num = int(exp_file.split('med')[0][-2:])
            for base_file2 in baseline_files2:
                base_num2 = int(base_file2.split('med')[0][-2:])
                if exp_num == base_num - 1 and exp_num == base_num2:
                    combinations.append((base_file, exp_file, base_file2))
    return combinations

def main():
    shifts = ["V", "M"]

    for shift in shifts:
        lab_dirs = {lab: glob.glob(os.path.join(root_folder, f"*{lab}")) for lab in labs}

        # Asegurarse de que cada laboratorio tenga directorio correspondiente
        if not all(lab_dirs.values()):
            print(f"Algunos laboratorios no tienen directorio para el turno {shift}.")
            continue

        # Si hay solo un laboratorio en la lista, graficar solo las gráficas individuales
        if len(labs) == 1:
            lab1 = labs[0]
            lab1_dir = lab_dirs[lab1][0] if lab_dirs[lab1] else None
            if not lab1_dir:
                print(f"Saltando laboratorio {lab1} por falta de directorio.")
                continue

            date_str = os.path.basename(lab1_dir).split(' ')[0]

            # Definir las rutas para baseline y experimental del laboratorio
            baseline_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Moduli_Curl_Baseline")
            exp_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Moduli_Curl_Experimental_Color")

            if not (os.path.exists(baseline_dir1) and os.path.exists(exp_dir1)):
                print(f"Saltando laboratorio {lab1} por falta de directorios.")
                continue

            # Directorio para gráficos individuales
            individual_output_dir1 = os.path.join(root_folder, f"Data Analysis", "Moduli Comparisons", lab1, shift)
            os.makedirs(individual_output_dir1, exist_ok=True)

            # Obtener archivos de cada directorio
            baseline_files1 = [f for f in os.listdir(baseline_dir1) if f.startswith('magnitude_') and f.endswith('mednegra.txt')]
            experimental_files1 = [f for f in os.listdir(exp_dir1) if f.startswith('magnitude_')]
            baseline_files2_1 = baseline_files1[1:] + [baseline_files1[0]] if baseline_files1 else []  # Rotar para n+1

            # Comprobar si hay suficientes archivos para generar combinaciones
            if not (baseline_files1 and experimental_files1):
                print(f"Saltando laboratorio {lab1} por falta de archivos.")
                continue

            combinations1 = generate_combinations(baseline_files1, experimental_files1, baseline_files2_1)

            # Graficar y guardar gráficos individuales
            for base_file1, exp_file1, base_file1_2 in combinations1:
                baseline_file1 = os.path.join(baseline_dir1, base_file1)
                experimental_file1 = os.path.join(exp_dir1, exp_file1)
                baseline_file1_2 = os.path.join(baseline_dir1, base_file1_2)

                if all(os.path.exists(file) for file in [baseline_file1, experimental_file1, baseline_file1_2]):
                    plot_individual(lab1, baseline_file1, experimental_file1, baseline_file1_2, individual_output_dir1, shift, date_str)

        else:
            # Iterar sobre cada combinación de laboratorios
            for i, lab1 in enumerate(labs):
                for lab2 in labs[i + 1:]:
                    # Obtener directorios para los laboratorios en turno
                    lab1_dir = lab_dirs[lab1][0] if lab_dirs[lab1] else None
                    lab2_dir = lab_dirs[lab2][0] if lab_dirs[lab2] else None

                    # Si alguno de los directorios no existe, saltar la iteración
                    if not lab1_dir or not lab2_dir:
                        print(f"Saltando combinación {lab1} vs {lab2} por falta de directorios.")
                        continue

                    date_str = os.path.basename(lab1_dir).split(' ')[0]

                    # Definir las rutas para baseline y experimental de cada laboratorio
                    baseline_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Moduli_Curl_Baseline")
                    exp_dir1 = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Moduli_Curl_Experimental_Color")

                    baseline_dir2 = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Moduli_Curl_Baseline")
                    exp_dir2 = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Moduli_Curl_Experimental_Color")

                    # Comprobar si las rutas existen antes de intentar acceder a ellas
                    if not (os.path.exists(baseline_dir1) and os.path.exists(exp_dir1) and os.path.exists(baseline_dir2) and os.path.exists(exp_dir2)):
                        print(f"Saltando combinación {lab1} vs {lab2} por falta de directorios.")
                        continue

                    # Directorio de salida para gráficos
                    output_directory = os.path.join(root_folder, "Data Analysis", "Moduli Comparisons", f"{lab1}_vs_{lab2}", shift)
                    os.makedirs(output_directory, exist_ok=True)

                    # Directorio para gráficos individuales
                    individual_output_dir1 = os.path.join(root_folder, f"Data Analysis", "Moduli Comparisons", lab1)
                    individual_output_dir2 = os.path.join(root_folder, f"Data Analysis", "Moduli Comparisons", lab2)
                    os.makedirs(individual_output_dir1, exist_ok=True)
                    os.makedirs(individual_output_dir2, exist_ok=True)

                    # Obtener archivos de cada directorio
                    baseline_files1 = [f for f in os.listdir(baseline_dir1) if f.startswith('magnitude_') and f.endswith('mednegra.txt')]
                    experimental_files1 = [f for f in os.listdir(exp_dir1) if f.startswith('magnitude_')]
                    baseline_files2_1 = baseline_files1[1:] + [baseline_files1[0]] if baseline_files1 else []  # Rotar para n+1

                    baseline_files2 = [f for f in os.listdir(baseline_dir2) if f.startswith('magnitude_') and f.endswith('mednegra.txt')]
                    experimental_files2 = [f for f in os.listdir(exp_dir2) if f.startswith('magnitude_')]
                    baseline_files2_2 = baseline_files2[1:] + [baseline_files2[0]] if baseline_files2 else []  # Rotar para n+1

                    # Comprobar si hay suficientes archivos para generar combinaciones
                    if not (baseline_files1 and experimental_files1 and baseline_files2 and experimental_files2):
                        print(f"Saltando combinación {lab1} vs {lab2} por falta de archivos.")
                        continue

                    combinations1 = generate_combinations(baseline_files1, experimental_files1, baseline_files2_1)
                    combinations2 = generate_combinations(baseline_files2, experimental_files2, baseline_files2_2)

                    # Comparar combinaciones entre los dos laboratorios
                    for (base_file1, exp_file1, base_file1_2), (base_file2, exp_file2, base_file2_2) in zip(combinations1, combinations2):
                        baseline_file1 = os.path.join(baseline_dir1, base_file1)
                        experimental_file1 = os.path.join(exp_dir1, exp_file1)
                        baseline_file1_2 = os.path.join(baseline_dir1, base_file1_2)

                        baseline_file2 = os.path.join(baseline_dir2, base_file2)
                        experimental_file2 = os.path.join(exp_dir2, exp_file2)
                        baseline_file2_2 = os.path.join(baseline_dir2, base_file2_2)

                        # Graficar comparación entre laboratorios
                        if all(os.path.exists(file) for file in [baseline_file1, experimental_file1, baseline_file1_2, baseline_file2, experimental_file2, baseline_file2_2]):
                            plot_triad_comparison(baseline_file1, experimental_file1, baseline_file1_2,
                                                  baseline_file2, experimental_file2, baseline_file2_2,
                                                  lab1, lab2, output_directory, shift, date_str)

                            # Graficar y guardar gráficos individuales
                            plot_individual(lab1, baseline_file1, experimental_file1, baseline_file1_2, individual_output_dir1, shift, date_str)
                            plot_individual(lab2, baseline_file2, experimental_file2, baseline_file2_2, individual_output_dir2, shift, date_str)

if __name__ == "__main__":
    main()

## 8

$$

    \Huge \text{Histograma Derivadas}

$$

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import re

def read_sensor_data(filepath):
    data = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except UnicodeDecodeError:
        print(f"Unicode decode error in file {filepath}")
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return np.array(data).T

def analyze_folder(root_folder):
    data_per_file = {}
    print(f"Analizando la carpeta raíz: {root_folder}\n")
    for root, dirs, files in os.walk(root_folder):
        dirs[:] = [d for d in dirs if d != "Data Analysis"]
        print(f"Subdirectorio: {root}")
        for filename in files:
            if filename.endswith(("negra.txt", "roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt")) and not filename.startswith("00"):
                file_path = os.path.join(root, filename)
                data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = data
                    print(f"   Encontrado y procesado: {filename}")
                else:
                    print(f"   Archivo vacío o no procesable: {filename}")
            else:
                print(f"   Archivo ignorado: {filename}")
        print("\n")
    return data_per_file

def smooth_data(data, window_length=10, polyorder=3):
    if len(data) < window_length:
        window_length = len(data) - (len(data) % 2 == 0)
    return savgol_filter(data, window_length=window_length, polyorder=polyorder)

def plot_comparison_grid(data_per_file, output_folder, title_info, lab):
    suffix_to_color = {
        "negra.txt": "black",
        "roja.txt": "red",
        "morada.txt": "purple",
        "azul.txt": "blue",
        "verde.txt": "green",
        "amarilla.txt": "yellow",
        "naranja.txt": "orange"
    }
    component_names = ["FRONT", "REAR", "LEFT", "RIGHT", "TOP", "BOTTOM"]

    min_length = min((len(data[0]) for data in data_per_file.values() if data.size > 0), default=None)
    if min_length is None:
        print("No se encontraron datos válidos para procesar.")
        return

    for base_key in sorted(data_per_file.keys()):
        if "negra.txt" in base_key:
            n = int(base_key.split('med')[0])
            base_data_before = data_per_file[base_key][:, :min_length]
            for exp_suffix in ["roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt"]:
                exp_key = f"{n:02d}med{exp_suffix}"
                base_key_after = f"{n+1:02d}mednegra.txt"

                if exp_key in data_per_file and base_key_after in data_per_file:
                    exp_data = data_per_file[exp_key][:, :min_length]
                    base_data_after = data_per_file[base_key_after][:, :min_length]

                    fig, axes = plt.subplots(6, 1, figsize=(15, 35), dpi=300)
                    for i, component in enumerate(component_names):
                        ax = axes[i]
                        smoothed_base_before = smooth_data(base_data_before[i])
                        smoothed_exp = smooth_data(exp_data[i])
                        smoothed_base_after = smooth_data(base_data_after[i])
                        derivative_base_before = np.gradient(smoothed_base_before)
                        derivative_exp = np.gradient(smoothed_exp)
                        derivative_base_after = np.gradient(smoothed_base_after)

                        # Histograma convencional
                        ax.hist(derivative_base_before, bins=30, color='gray', alpha=0.6, label=f'Base Before', edgecolor='black')
                        ax.hist(derivative_exp, bins=30, color=suffix_to_color[exp_suffix], alpha=0.6, label=f'Experimental', edgecolor='black')
                        ax.hist(derivative_base_after, bins=30, color='black', alpha=0.6, label=f'Base After', edgecolor='black')

                        ax.set_title(f"{component} Component", fontsize=16)
                        ax.set_xlabel("Derivative Value", fontsize=14)
                        ax.set_ylabel("Frequency", fontsize=14)
                        ax.legend(fontsize=12)
                        ax.grid(True)

                    date, shift = title_info
                    shift_full = "MORNING" if shift == "M" else "AFTERNOON"
                    plt.suptitle(f"Histograms - {date} - {shift_full} - {lab}", fontsize=20)
                    plt.tight_layout(pad=5.0)

                    output_file = os.path.join(output_folder, f"histogram_{n:02d}_{exp_suffix.split('.')[0]}.pdf")
                    plt.savefig(output_file, bbox_inches='tight', format='png')
                    plt.close()

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def main():
    turnos = ["M", "V"]

    for lab in labs:
        for turno in turnos:
            experiment_folder, date_str = find_experiment_folder(root_folder, lab, turno)
            if not experiment_folder:
                print(f"No se encontró ningún directorio para {lab}, {turno}.")
                continue

            output_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "Comparison_Histograms", turno)
            os.makedirs(output_folder, exist_ok=True)

            title_info = (date_str, turno)

            data_per_file = analyze_folder(experiment_folder)
            plot_comparison_grid(data_per_file, output_folder, title_info, lab)

if __name__ == "__main__":
    main()


## 9

$$

    \Huge \text{Comparaciones Laser}

$$

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import re

def read_sensor_data(filepath):
    data = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except UnicodeDecodeError:
        print(f"Unicode decode error in file {filepath}")
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return np.array(data).T

def analyze_folder(root_folder):
    data_per_file = {}
    for root, dirs, files in os.walk(root_folder):
        dirs[:] = [d for d in dirs if d != "Data Analysis"]
        for filename in files:
            if filename.endswith(("negra.txt", "roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt")) and not filename.startswith("00"):
                file_path = os.path.join(root, filename)
                data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = data
    return data_per_file

def smooth_data(data, window_length=10, polyorder=3):
    if len(data) < window_length:
        window_length = len(data) - (len(data) % 2 == 0)
    return savgol_filter(data, window_length=window_length, polyorder=polyorder)

def plot_comparison_grid(data_per_file, output_folder, title_info, lab):
    suffix_to_color = {
        "negra.txt": "black",
        "roja.txt": "red",
        "morada.txt": "purple",
        "azul.txt": "blue",
        "verde.txt": "green",
        "amarilla.txt": "yellow",
        "naranja.txt": "orange"
    }

    component_names = ["FRONT", "REAR", "LEFT", "RIGHT", "TOP", "BOTTOM"]
    component_groups = [
        (["FRONT", "REAR"], "horizontal"),
        (["LEFT", "RIGHT"], "transversal"),
        (["TOP", "BOTTOM"], "vertical")
    ]

    for base_key in sorted(data_per_file.keys()):
        if "negra.txt" in base_key:
            try:
                n = int(re.search(r'\d+', base_key.split('med')[0]).group())
            except (ValueError, AttributeError):
                print(f"El archivo {base_key} no contiene un número válido para conversión.")
                continue

            base_data_before = data_per_file[base_key]

            for exp_suffix in ["roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt"]:
                exp_key = f"{n:02d}med{exp_suffix}"
                base_key_after = f"{n+1:02d}mednegra.txt"

                if exp_key in data_per_file and base_key_after in data_per_file:
                    exp_data = data_per_file[exp_key]
                    base_data_after = data_per_file[base_key_after]

                    for group_components, axis_name in component_groups:
                        fig, axes = plt.subplots(2, 2, figsize=(25, 18), dpi=300)
                        axes = axes.flatten()

                        for i, component in enumerate(group_components):
                            component_index = component_names.index(component)

                            ax_before = axes[i * 2]
                            ax_after = axes[i * 2 + 1]

                            smoothed_base_before = smooth_data(base_data_before[component_index])
                            smoothed_exp = smooth_data(exp_data[component_index])
                            smoothed_base_after = smooth_data(base_data_after[component_index])

                            ax_before.plot(smoothed_base_before, label=f'Base Line: {base_key}', color='black', linewidth=2)
                            ax_before.plot(smoothed_exp, label=f'Experimental: {exp_key}', color=suffix_to_color[exp_suffix], linewidth=2)
                            ax_before.set_title(f"Component {component} (Before)", fontsize=20)
                            ax_before.set_xlabel("Register", fontsize=20)
                            ax_before.set_ylabel("Volts", fontsize=20)
                            ax_before.tick_params(axis='both', which='major', labelsize=15)
                            ax_before.legend(fontsize=15)

                            ax_after.plot(smoothed_base_after, label=f'Base Line: {base_key_after}', color='black', linewidth=2)
                            ax_after.plot(smoothed_exp, label=f'Experimental: {exp_key}', color=suffix_to_color[exp_suffix], linewidth=2)
                            ax_after.set_title(f"Component {component} (After)", fontsize=20)
                            ax_after.set_xlabel("Register", fontsize=20)
                            ax_after.set_ylabel("Volts", fontsize=20)
                            ax_after.tick_params(axis='both', which='major', labelsize=15)
                            ax_after.legend(fontsize=15)

                        date, shift = title_info
                        shift_full = "MORNING" if shift == "M" else "AFTERNOON"
                        color_name = exp_suffix.split('.')[0].upper()

                        experiment_name = exp_key.split('med')[0] + exp_key.split('med')[1].split('.')[0]
                        output_file = os.path.join(
                            output_folder,
                            f"{axis_name}_{experiment_name}.png"
                        )

                        plt.suptitle(
                            f"COMPARISON BASE BEFORE {n:02d} AND AFTER {n+1:02d} INTERVENTION - {date} - {shift_full} - INTERVENTION: {color_name} - LABORATORY: {lab} - COMPONENT: {axis_name.upper()}",
                            fontsize=25
                        )
                        plt.tight_layout(rect=[0, 0, 1, 0.95])
                        plt.subplots_adjust(hspace=0.3, wspace=0.15)
                        plt.savefig(output_file, bbox_inches='tight', format='png')
                        plt.grid(True)
                        plt.close()

def find_experiment_folder(root_folder, lab, turno):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and turno in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def main():
    turnos = ["M", "V"]

    for lab in labs:
        for turno in turnos:
            experiment_folder, date_str = find_experiment_folder(root_folder, lab, turno)
            if not experiment_folder:
                print(f"No se encontró ningún directorio para {lab}, {turno}.")
                continue

            output_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "Comparison", turno)
            os.makedirs(output_folder, exist_ok=True)

            title_info = (date_str, turno)

            data_per_file = analyze_folder(experiment_folder)
            plot_comparison_grid(data_per_file, output_folder, title_info, lab)

if __name__ == "__main__":
    main()


## 11

$$

    \Huge \text{Topological Data Analysis}

$$

In [ ]:
import os
import numpy as np
import ruptures as rpt
import matplotlib.pyplot as plt

def detect_breakpoints(signal):
    if len(signal) < 2:
        print("Advertencia: La señal es demasiado corta para detectar puntos de cambio.")
        return []

    algo = rpt.Pelt(model="rbf").fit(signal)
    pen = max(1, 0.1 * len(signal))  # Penalización proporcional al tamaño de la señal
    try:
        return algo.predict(pen=pen)
    except rpt.exceptions.BadSegmentationParameters:
        print(f"Error: No se pudieron detectar puntos de cambio. Señal: {signal}, Penalización: {pen}")
        return []

def collect_statistics(folder):
    file_list = []
    means = []
    variances = []
    std_devs = []

    for filename in sorted(os.listdir(folder)):
        if filename.endswith(".txt"):
            filepath = os.path.join(folder, filename)
            try:
                data = read_sensor_data(filepath)
                if data.size > 0:
                    file_list.append(filename)
                    means.append(np.mean(data))
                    variances.append(np.var(data))
                    std_devs.append(np.std(data))
            except Exception as e:
                print(f"Error procesando {filename}: {e}")

    return file_list, means, variances, std_devs

def analyze_experiments(root_folder, labs):
    sessions = ["V", "M"]
    results = {session: {lab: {} for lab in labs} for session in sessions}

    for lab in labs:
        for session in sessions:
            experiment_folder = find_experiment_folder(root_folder, lab, session)
            if isinstance(experiment_folder, tuple):
                experiment_folder = experiment_folder[0]
            if not experiment_folder or not os.path.exists(experiment_folder):
                print(f"Advertencia: No se encontró carpeta para {lab} en sesión {session}.")
                continue

            output_folder = os.path.join(experiment_folder, "Metanalisis")
            os.makedirs(output_folder, exist_ok=True)

            file_list, means, variances, std_devs = collect_statistics(experiment_folder)

            plt.figure(figsize=(10, 6))
            plt.plot(means, marker="o", label="Medias")
            plt.title(f"Tendencia de Medias - {lab} ({session})")
            plt.xlabel("Mediciones")
            plt.ylabel("Media")
            plt.legend()
            plt.grid()
            plt.savefig(os.path.join(output_folder, f"{lab}_{session}_means_trend.png"))
            plt.close()

            with open(os.path.join(output_folder, f"{lab}_{session}_coeff_variation.txt"), "w") as f:
                f.write("Archivo, Coeficiente de Variación\n")
                for i, mean in enumerate(means):
                    if mean != 0:
                        cv = std_devs[i] / mean
                        f.write(f"{file_list[i]}, {cv}\n")

            breakpoints = detect_breakpoints(np.array(means))
            with open(os.path.join(output_folder, f"{lab}_{session}_breakpoints.txt"), "w") as f:
                f.write("Puntos de Cambio Detectados\n")
                f.write(str(breakpoints))

            results[session][lab] = {
                "file_list": file_list,
                "means": means,
                "variances": variances,
                "std_devs": std_devs,
                "breakpoints": breakpoints
            }

    return results

def process_all_experiments(root_folder):
    """
    Procesa todos los experimentos en los laboratorios y consolida los resultados.

    Parámetros:
    - root_folder: Carpeta raíz donde se encuentran los datos crudos.
    """
    results = analyze_experiments(root_folder, labs)
    output_folder = os.path.join(root_folder, "Metanalisis")
    os.makedirs(output_folder, exist_ok=True)

    with open(os.path.join(output_folder, "summary.txt"), "w") as f:
        for session, labs_results in results.items():
            for lab, res in labs_results.items():
                f.write(f"Laboratorio: {lab}, Sesión: {session}\n")
                breakpoints = res.get('breakpoints', [])  # Obtiene breakpoints o lista vacía si no existe
                f.write(f"Breakpoints: {breakpoints}\n")
                f.write("\n")

process_all_experiments(root_folder)


3D CURL

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import datetime
import os
import glob
from matplotlib.animation import FuncAnimation

# Función para leer datos del sensor
def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                try:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append([float(num) for num in parts[1].strip('()').split(',')])
                except ValueError as e:
                    print(f"Error processing line in file {filepath}: {line}\n{e}")
    return timestamps, np.array(values)

# Función para graficar un solo laboratorio
def plot_single_lab(baseline_file, exp_file, color, output_directory, shift, date_str, lab_name):
    try:
        _, baseline_values = read_data(baseline_file)
        _, exp_values = read_data(exp_file)
    except Exception as e:
        print(f"Error reading data files:\n{e}")
        return

    shift_label = "Afternoon" if shift == "V" else "Morning"

    # Generar el nombre del archivo de salida
    output_filename = f"{os.path.basename(baseline_file).replace('mednegra', f'comparison_{lab_name}')}_{os.path.basename(exp_file)}"
    output_filepath = os.path.join(output_directory, f"{output_filename}.gif")

    # Verificar si el archivo ya existe
    if os.path.exists(output_filepath):
        print(f"El archivo {output_filepath} ya existe. Se omite la generación del gráfico.")
        return

    # Crear la carpeta de salida si no existe
    os.makedirs(output_directory, exist_ok=True)

    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(baseline_values[:, 0], baseline_values[:, 1], baseline_values[:, 2], c='black', label=os.path.basename(baseline_file), s=50, alpha=0.6)
    ax.scatter(exp_values[:, 0], exp_values[:, 1], exp_values[:, 2], c=color, label=os.path.basename(exp_file), s=50, alpha=0.6)
    ax.set_title(f'{lab_name} - {shift_label} Shift - Date: {date_str}', fontsize=20)
    ax.set_xlabel('VOLTS', fontsize=15)
    ax.set_ylabel('VOLTS', fontsize=15)
    ax.set_zlabel('VOLTS', fontsize=15)
    legend = ax.legend(fontsize=14, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend.legend_handles:
        handle.set_sizes([300])

    def update(frame):
        ax.view_init(elev=10, azim=frame)
        return fig,

    ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=100, blit=False)
    ani.save(output_filepath, writer='pillow', fps=10)
    print(f"Saved animation as {output_filepath}")
    plt.close(fig)

def plot_comparison_labs(baseline_file_lab1, exp_file_lab1, color_lab1, baseline_file_lab2, exp_file_lab2, color_lab2, output_directory, shift, date_str, lab1_name, lab2_name):
    try:
        _, baseline_values_lab1 = read_data(baseline_file_lab1)
        _, exp_values_lab1 = read_data(exp_file_lab1)
        _, baseline_values_lab2 = read_data(baseline_file_lab2)
        _, exp_values_lab2 = read_data(exp_file_lab2)
    except Exception as e:
        print(f"Error reading data files:\n{e}")
        return

    shift_label = "Afternoon" if shift == "V" else "Morning"

    # Generar el nombre del archivo de salida
    output_filename = f"{os.path.basename(baseline_file_lab1).replace('mednegra', f'comparison_{lab1_name}_vs_{lab2_name}')}_{os.path.basename(exp_file_lab1)}"
    output_filepath = os.path.join(output_directory, f"{output_filename}.gif")

    # Verificar si el archivo ya existe
    if os.path.exists(output_filepath):
        print(f"El archivo {output_filepath} ya existe. Se omite la generación del gráfico.")
        return

    # Crear la carpeta de salida si no existe
    os.makedirs(output_directory, exist_ok=True)

    fig = plt.figure(figsize=(24, 10))

    # Subgráfica para el laboratorio 1
    ax_lab1 = fig.add_subplot(121, projection='3d')
    ax_lab1.scatter(baseline_values_lab1[:, 0], baseline_values_lab1[:, 1], baseline_values_lab1[:, 2], c='black', label=os.path.basename(baseline_file_lab1), s=50, alpha=0.6)
    ax_lab1.scatter(exp_values_lab1[:, 0], exp_values_lab1[:, 1], exp_values_lab1[:, 2], c=color_lab1, label=os.path.basename(exp_file_lab1), s=50, alpha=0.6)
    ax_lab1.set_title(f'{lab1_name} - {shift_label} Shift - Date: {date_str}', fontsize=20)
    ax_lab1.set_xlabel('VOLTS', fontsize=15)
    ax_lab1.set_ylabel('VOLTS', fontsize=15)
    ax_lab1.set_zlabel('VOLTS', fontsize=15)
    legend_lab1 = ax_lab1.legend(fontsize=14, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend_lab1.legend_handles:
        handle.set_sizes([300])

    # Subgráfica para el laboratorio 2
    ax_lab2 = fig.add_subplot(122, projection='3d')
    ax_lab2.scatter(baseline_values_lab2[:, 0], baseline_values_lab2[:, 1], baseline_values_lab2[:, 2], c='black', label=os.path.basename(baseline_file_lab2), s=50, alpha=0.6)
    ax_lab2.scatter(exp_values_lab2[:, 0], exp_values_lab2[:, 1], exp_values_lab2[:, 2], c=color_lab2, label=os.path.basename(exp_file_lab2), s=50, alpha=0.6)
    ax_lab2.set_title(f'{lab2_name} - {shift_label} Shift - Date: {date_str}', fontsize=20)
    ax_lab2.set_xlabel('VOLTS', fontsize=15)
    ax_lab2.set_ylabel('VOLTS', fontsize=15)
    ax_lab2.set_zlabel('VOLTS', fontsize=15)
    legend_lab2 = ax_lab2.legend(fontsize=14, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend_lab2.legend_handles:
        handle.set_sizes([300])

    def update(frame):
        ax_lab1.view_init(elev=10, azim=frame)
        ax_lab2.view_init(elev=10, azim=frame)
        return fig,

    ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=100, blit=False)
    ani.save(output_filepath, writer='pillow', fps=10)
    print(f"Saved animation as {output_filepath}")
    plt.close(fig)

# Función principal para procesar los archivos de todos los laboratorios
def main(root_folder, labs):
    date_str = os.path.basename(root_folder)
    date_str = date_str.replace("-2", "").strip()
    shifts = ["V", "M"]

    color_map = {
        "medroja.txt": "red",
        "medmorada.txt": "purple",
        "medazul.txt": "blue",
        "medverde.txt": "green",
        "medamarilla.txt": "yellow",
        "mednaranja.txt": "orange"
    }

    # Verificar si hay solo un laboratorio
    if len(labs) == 1:
        lab = labs[0]  # Único laboratorio
        lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))

        if not lab_dirs:
            print(f"No se encontró el directorio para {lab}.")
            return

        lab_dir = lab_dirs[0]

        for shift in shifts:
            lab_baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Curl_Baseline")
            lab_exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

            output_directory = os.path.join(root_folder, f"Data Analysis", f"3D Curl Comparisons", shift)
            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            # Omitir si alguna de las carpetas requeridas no existe
            if not os.path.exists(lab_baseline_dir) or not os.path.exists(lab_exp_dir):
                print(f"Omitiendo cálculo para el turno {shift} ya que uno o más directorios no existen en {lab}.")
                continue

            lab_baseline_files = [f for f in os.listdir(lab_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
            lab_exp_files = [f for f in os.listdir(lab_exp_dir) if any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]

            # Verificar que hay archivos para procesar
            if not lab_baseline_files or not lab_exp_files:
                print(f"Omitiendo cálculo para el turno {shift} ya que no se encontraron archivos de base o experimentales en {lab}.")
                continue

            combinations_lab = generate_combinations(lab_baseline_files, lab_exp_files)

            # Comparar internamente las combinaciones del único laboratorio
            for base_file, exp_file in combinations_lab:
                baseline_file_lab = os.path.join(lab_baseline_dir, base_file)
                experimental_file_lab = os.path.join(lab_exp_dir, exp_file)

                if os.path.exists(baseline_file_lab) and os.path.exists(experimental_file_lab):
                    exp_suffix_lab = ''.join([i for i in os.path.basename(experimental_file_lab).split('curl_')[-1] if not i.isdigit()])
                    color_lab = color_map.get(exp_suffix_lab, 'black')
                    # Llamar a la función para graficar un solo laboratorio
                    plot_single_lab(baseline_file_lab, experimental_file_lab, color_lab, output_directory, shift, date_str, lab)

    else:
        # Combinaciones de laboratorios cuando hay más de un laboratorio
        lab_combinations = [(labs[i], labs[j]) for i in range(len(labs)) for j in range(i+1, len(labs))]

        for shift in shifts:
            for lab1, lab2 in lab_combinations:
                lab1_dirs = glob.glob(os.path.join(root_folder, f"*{lab1}"))
                lab2_dirs = glob.glob(os.path.join(root_folder, f"*{lab2}"))

                if not lab1_dirs:
                    print(f"No se encontró el directorio para {lab1}.")
                    continue
                if not lab2_dirs:
                    print(f"No se encontró el directorio para {lab2}.")
                    continue

                lab1_dir = lab1_dirs[0]
                lab2_dir = lab2_dirs[0]

                lab1_baseline_dir = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Curl_Baseline")
                lab1_exp_dir = os.path.join(lab1_dir, f"{date_str}.{shift} - {lab1}", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

                lab2_baseline_dir = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Curl_Baseline")
                lab2_exp_dir = os.path.join(lab2_dir, f"{date_str}.{shift} - {lab2}", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

                output_directory = os.path.join(root_folder, f"Data Analysis", "3D Curl Comparisons", f"{lab1}_vs_{lab2}", shift)
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory)

                # Omitir si alguna de las carpetas requeridas no existe
                if not os.path.exists(lab1_baseline_dir) or not os.path.exists(lab1_exp_dir) or not os.path.exists(lab2_baseline_dir) or not os.path.exists(lab2_exp_dir):
                    print(f"Omitiendo cálculo para el turno {shift} y combinación {lab1} vs {lab2} ya que uno o más directorios no existen.")
                    continue

                lab1_baseline_files = [f for f in os.listdir(lab1_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
                lab2_baseline_files = [f for f in os.listdir(lab2_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]

                lab1_exp_files = [f for f in os.listdir(lab1_exp_dir) if any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]
                lab2_exp_files = [f for f in os.listdir(lab2_exp_dir) if any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]

                # Verificar que hay archivos para procesar
                if not lab1_baseline_files or not lab2_baseline_files or not lab1_exp_files or not lab2_exp_files:
                    print(f"Omitiendo cálculo para el turno {shift} y combinación {lab1} vs {lab2} ya que no se encontraron archivos de base o experimentales en uno de los laboratorios.")
                    continue

                combinations_lab1 = generate_combinations(lab1_baseline_files, lab1_exp_files)
                combinations_lab2 = generate_combinations(lab2_baseline_files, lab2_exp_files)

                # Asumiendo que las combinaciones tienen el mismo número de pares para lab1 y lab2
                for (base_lab1, exp_lab1), (base_lab2, exp_lab2) in zip(combinations_lab1, combinations_lab2):
                    baseline_file_lab1 = os.path.join(lab1_baseline_dir, base_lab1)
                    experimental_file_lab1 = os.path.join(lab1_exp_dir, exp_lab1)
                    baseline_file_lab2 = os.path.join(lab2_baseline_dir, base_lab2)
                    experimental_file_lab2 = os.path.join(lab2_exp_dir, exp_lab2)

                    if os.path.exists(baseline_file_lab1) and os.path.exists(experimental_file_lab1) and os.path.exists(baseline_file_lab2) and os.path.exists(experimental_file_lab2):
                        exp_suffix_lab1 = ''.join([i for i in os.path.basename(experimental_file_lab1).split('curl_')[-1] if not i.isdigit()])
                        exp_suffix_lab2 = ''.join([i for i in os.path.basename(experimental_file_lab2).split('curl_')[-1] if not i.isdigit()])
                        color_lab1 = color_map.get(exp_suffix_lab1, 'black')
                        color_lab2 = color_map.get(exp_suffix_lab2, 'black')
                        plot_comparison_labs(baseline_file_lab1, experimental_file_lab1, color_lab1, baseline_file_lab2, experimental_file_lab2, color_lab2, output_directory, shift, date_str, lab1, lab2)

# Generar combinaciones de archivos de línea base y experimentales
def generate_combinations(baseline_files, exp_files):
    combinations = []
    for base_file in baseline_files:
        base_num = int(base_file.split('med')[0][-2:])
        for exp_file in exp_files:
            exp_num = int(exp_file.split('med')[0][-2:])
            if exp_num == base_num or exp_num == base_num - 1:
                combinations.append((base_file, exp_file))
    return combinations

if __name__ == "__main__":
    main(root_folder, labs)

Codigo Latex

In [ ]:
#@siva se añade codigo de @alcrlp el cual en base a la jornada genera un reporte usando Latex 
import os
import numpy as np
from scipy.stats import kurtosis, skew, iqr


subsecciones_personalizadas = {
    "Sum_Absolute_Helicity_Comp": "Suma acumulada de valores absolutos de la Helicidad",
    "Sum_Enstrophy_Comp": "Suma acumulada de la Enstrofia",
    "Sum_Helicity_Comp": "Suma acumulada de la Helicidad"
}

def add_gravity_behavior_section(file, root_folder, labs):
    """
    Agrega una sección "Gravity Behavior" al documento LaTeX, accediendo a las imágenes desde la carpeta "Pesajes/graficas".
    
    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - root_folder: directorio raíz que contiene las carpetas.
    - labs: lista de laboratorios (e.g., ["Lab_Betta", "Lab_Gamma"]).
    """
    shifts = ["M", "V"]  # Turnos: M = Matutino, V = Vespertino
    date = os.path.basename(root_folder)  # Extraer la fecha desde el root_folder

    # Crear la sección principal
    file.write("\\section{Gravity Behavior}\n")

    for lab in labs:
        file.write(f"\\subsection{{Lab \\textit{{{lab}}}}}\n")
        for shift in shifts:
            # Construir la ruta de la carpeta de imágenes
            output_folder = os.path.join(root_folder, f"{date} - {lab}", f"{date}.{shift} - {lab}", "Pesajes", "graficas")
            
            if os.path.exists(output_folder):
                file.write(f"\\subsubsection{{Turno {'Matutino' if shift == 'M' else 'Vespertino'}}}\n")

                # Listar las imágenes PNG en la carpeta
                images = [f for f in os.listdir(output_folder) if f.endswith('.png')]
                if not images:
                    continue  # Si no hay imágenes, pasar al siguiente turno

                for image in images:
                    image_path = os.path.join(output_folder, image).replace(os.sep, '/')
                    caption = f"Gravity Behavior: {os.path.splitext(image)[0].replace('_', ' ').capitalize()}"
                    label = f"fig:gravity_behavior_{lab}_{shift}_{os.path.splitext(image)[0]}"

                    # Incluir la imagen en el documento LaTeX
                    file.write("\\begin{figure}[H]\n")
                    file.write(f"    \\centering\n")
                    file.write(f"    \\includegraphics[width=0.9\\textwidth]{{{image_path}}}\n")
                    file.write(f"    \\caption{{{caption}}}\n")
                    file.write(f"    \\label{{{label}}}\n")
                    file.write("\\end{figure}\n\n")



def analyze_folder(root_folder):
    """
    Recorre una carpeta y recopila datos de archivos específicos.
    """
    data_per_file = {}
    for root, dirs, files in os.walk(root_folder):
        if 'Data Analysis' in dirs:
            dirs.remove('Data Analysis')

        for filename in files:
            if filename.endswith(("negra.txt", "roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt")):
                file_path = os.path.join(root, filename)
                timestamps, data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = (timestamps, data, file_path)
    return data_per_file


# Función para leer datos de los sensores
def read_sensor_data(filepath):
    """
    Lee datos de sensores desde un archivo.
    """
    data = []
    timestamps = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return timestamps, np.array(data).T

# Función para generar estadísticas de datos
def generate_statistics(data):
    """
    Genera estadísticas para un conjunto de datos.
    """
    return {
        'mean': np.mean(data, axis=1),
        'median': np.median(data, axis=1),
        'variance': np.var(data, axis=1),
        'stdDev': np.std(data, axis=1),
        'min': np.min(data, axis=1),
        'max': np.max(data, axis=1),
        'firstQuartile': np.percentile(data, 25, axis=1),
        'thirdQuartile': np.percentile(data, 75, axis=1),
        'iqr': iqr(data, axis=1),
        'trimmedVariance': [np.var(np.trim_zeros(np.sort(sensor_data))) for sensor_data in data],
        'kurtosis': kurtosis(data, axis=1, fisher=True, bias=False),
        'skewness': skew(data, axis=1, bias=False)
    }

def generate_latex_statistics(file, root_folder, labs):
    """
    Genera estadísticas de archivos en carpetas específicas y las escribe en un archivo LaTeX.
    
    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - root_folder: directorio raíz que contiene las carpetas.
    - labs: lista de laboratorios (e.g., ["Lab_Betta", "Lab_Gamma"]).
    """
    shifts = ["M", "V"]  # Turnos: M = Matutino, V = Vespertino
    date = os.path.basename(root_folder)  # Extraer la fecha desde el root_folder

    file.write("\\section{General Statistics}\n")

    laser_components = ["FRONT", "REAR", "LEFT", "RIGHT", "TOP", "BOTTOM"]

    for lab in labs:
        file.write(f"\\subsection{{\\textit{{{lab}}}}}\n")
        for shift in shifts:
            # Construir la ruta dinámica para cada laboratorio y turno
            experiment_folder = os.path.join(root_folder, f"{date} - {lab}", f"{date}.{shift} - {lab}")
            
            if os.path.exists(experiment_folder):
                file.write(f"\\subsubsection{{Shift {'Morning' if shift == 'M' else 'Evening'}}}\n")

                # Analizar los archivos en la carpeta
                data_per_file = analyze_folder(experiment_folder)
                if not data_per_file:
                    file.write("No se encontraron archivos para analizar.\n")
                    continue

                # Ordenar los archivos por patrón de medición
                sorted_files = sorted(data_per_file.keys(), key=lambda x: ("LB" not in x, x))

                # Generar una tabla por láser
                for i, component in enumerate(laser_components):
                    file.write("\\begin{table}[H]\n")
                    file.write("\\centering\n")
                    file.write("\\resizebox{\\textwidth}{!}{\n")
                    file.write("\\begin{tabular}{p{2.7cm}" + "p{1.5cm} " * 12 + "}\n")
                    file.write("\\toprule\n")
                    file.write("\\textbf{FILENAME} & \\textbf{Median} & \\textbf{Mean} & \\textbf{Variance} & \\textbf{STD} & \\textbf{Min} & \\textbf{Max} & \\textbf{First Quartile} & \\textbf{Third Quartile} & \\textbf{IQR} & \\textbf{Trimmed Variance} & \\textbf{Kurtosis} & \\textbf{Skewness} \\\\\n")
                    file.write("\\midrule\n")

                    for filename in sorted_files:
                        timestamps, data, _ = data_per_file[filename]
                        stats = generate_statistics(data)

                        # Escribir una fila por archivo para el láser actual
                        row = (
                            f"{filename} & "
                            f"{stats['median'][i]:.4f} & {stats['mean'][i]:.4f} & {stats['variance'][i]:.4e} & "
                            f"{stats['stdDev'][i]:.4f} & {stats['min'][i]:.4f} & {stats['max'][i]:.4f} & "
                            f"{stats['firstQuartile'][i]:.4f} & {stats['thirdQuartile'][i]:.4f} & {stats['iqr'][i]:.4f} & "
                            f"{stats['trimmedVariance'][i]:.4e} & {stats['kurtosis'][i]:.4f} & {stats['skewness'][i]:.4f} \\\\\n"
                        )
                        file.write(row)

                    # Cerrar la tabla
                    file.write("\\bottomrule\n")
                    file.write("\\end{tabular}}\n")
                    file.write(f"\\caption{{Estadísticas del láser {component} en {lab}, turno {'Matutino' if shift == 'M' else 'Vespertino'}}}\n")
                    file.write(f"\\label{{tab:{lab}_{shift}_{component}}}\n")
                    file.write("\\end{table}\n\n")


# Nueva función para extraer la fecha automáticamente del root_folder
def extraer_fecha(root_folder):
    """
    Extrae la fecha del root_folder automáticamente y devuelve el formato adecuado.
    Ejemplo: si el root_folder contiene '08Dic24', retorna '08Dic24'.
    """
    folder_name = os.path.basename(root_folder)
    for part in folder_name.split():
        if len(part) == 7 and part[:2].isdigit():
            return part
    raise ValueError("No se encontró una fecha válida en el root_folder.")


# Nueva función para procesar las series de tiempo comparativas
def procesar_comparison_times_series(file, paths_por_laboratorio):
    """
    Crea una sección en el documento LaTeX para mostrar series de tiempo comparativas.
    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - paths_por_laboratorio: dict con {nombre_laboratorio: rutas_comparison}
    """
    
    # Crear la sección principal
    escribir_seccion(file, "Comparison Times Series", "section")
    
    # Iterar sobre cada laboratorio
    for lab_name, lab_paths in paths_por_laboratorio.items():
        # Crear una subsección para el laboratorio
        escribir_seccion(file, lab_name, "subsection")
        
        # Iterar sobre carpetas M y V
        for subfolder in lab_paths:
            if os.path.exists(subfolder):
                image_files = [f for f in os.listdir(subfolder) if f.endswith(".png")]

                for image_file in image_files:
                    image_path = os.path.join(subfolder, image_file).replace(os.sep, '/')
                    caption = f"{os.path.splitext(image_file)[0].replace('_', ' ').capitalize()}"
                    label = f"fig:{os.path.splitext(image_file)[0]}"

                    # Agregar la figura al documento LaTeX
                    agregar_figura(file, image_path, caption, label)

fecha = extraer_fecha(root_folder)                    


labs = {
    "Lab_Betta": [
        os.path.join(root_folder, f"{fecha} - Lab_Betta", f"{fecha}.M - Lab_Betta", "Data Analysis", "Graphics", "Comparison", "M"),
        os.path.join(root_folder, f"{fecha} - Lab_Betta", f"{fecha}.M - Lab_Betta", "Data Analysis", "Graphics", "Comparison", "V")
    ],
    "Lab_Gamma": [
        os.path.join(root_folder, f"{fecha} - Lab_Gamma", f"{fecha}.M - Lab_Gamma", "Data Analysis", "Graphics", "Comparison", "M"),
        os.path.join(root_folder, f"{fecha} - Lab_Gamma", f"{fecha}.M - Lab_Gamma", "Data Analysis", "Graphics", "Comparison", "V")
    ]
}


def procesar_3d_curl(file, root_folder, shift):
    """
    Procesa las imágenes de la sección "3D Curl" según el turno (matutino o vespertino).
    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - root_folder: ruta raíz del evento.
    - shift: turno ('M' para matutino o 'V' para vespertino).
    """
    # Construir la ruta de las imágenes
    curl_folder = os.path.join(root_folder, "Data Analysis", "3D_Curl_images", "Lab_Betta_vs_Lab_Gamma", shift)

    # Crear la sección principal
    escribir_seccion(file, "3D Curl", "section")

    if os.path.exists(curl_folder):
        image_files = [f for f in os.listdir(curl_folder) if f.endswith(".png")]

        for image_file in image_files:
            image_path = os.path.join(curl_folder, image_file).replace(os.sep, '/')
            caption = f"{os.path.splitext(image_file)[0].replace('_', ' ').capitalize()}"
            label = f"fig:{os.path.splitext(image_file)[0]}"

            # Agregar la figura al documento LaTeX
            agregar_figura(file, image_path, caption, label)
    else:
        print(f"La carpeta {curl_folder} no existe.")






# Función para extraer la fecha del root_folder
def extraer_fecha_de_ruta(root_folder):
    # Obtener el nombre de la última carpeta de la ruta
    folder_name = os.path.basename(root_folder)

    # Supongamos que la fecha siempre tiene el formato "DDMesYY", como "16Ago24"
    dia = folder_name[:2]       # Primeros dos caracteres para el día
    mes = folder_name[2:5]      # Caracteres 3 a 5 para el mes (e.g., "Ago")
    año = folder_name[5:]       # Resto para el año (e.g., "24")

    # Formatear la fecha en el estilo requerido, por ejemplo: "16 de Agosto de 2024"
    meses = {"Ene": "Enero", "Feb": "Febrero", "Mar": "Marzo", "Abr": "Abril",
             "May": "Mayo", "Jun": "Junio", "Jul": "Julio", "Ago": "Agosto",
             "Sep": "Septiembre", "Oct": "Octubre", "Nov": "Noviembre", "Dic": "Diciembre"}
    
    mes_completo = meses.get(mes, mes)  # Obtener el nombre completo del mes

    # Asumir que el año es 20YY (e.g., "24" corresponde a 2024)
    año_completo = f"20{año}"

    return f"{dia} de {mes_completo} de {año_completo}"


# Función para recorrer recursivamente todas las subcarpetas
def recorrer_subcarpetas(carpeta):
    subcarpetas_encontradas = []
    for root, dirs, files in os.walk(carpeta):
        for directory in dirs:
            subcarpetas_encontradas.append(os.path.join(root, directory))
    return subcarpetas_encontradas

# Función para verificar y crear carpetas si no existen
def crear_carpeta(ruta_carpeta):
    try:
        os.makedirs(ruta_carpeta, exist_ok=True)
        print(f"Carpeta creada o ya existente: {ruta_carpeta}")
    except Exception as e:
        print(f"Error al crear la carpeta {ruta_carpeta}: {e}")

# Función para escribir el preámbulo del archivo LaTeX
def escribir_preambulo(file, root_folder):
    # Extraer la fecha desde la ruta
    fecha_jornada = extraer_fecha_de_ruta(root_folder)
    
    preambulo = [
        "\\documentclass{article}\n",
        "\\usepackage{amsmath}\n",
        "\\usepackage{graphicx} % Required for inserting images\n",
        "\\usepackage{listings}\n",
        "\\usepackage{xcolor}\n",
        "\\usepackage{float}\n",
        "\\usepackage[utf8]{inputenc}\n",
        "\\usepackage[T1]{fontenc}\n",
        "\\usepackage{booktabs}\n",
        "\\usepackage{geometry}\n",
        "\\geometry{a4paper, margin=1in}\n",
        f"\\title{{Jornada del día {fecha_jornada}}}\n",  # Usar la fecha extraída en el título
        "\\author{Equipo de Análisis}\n",
        "\\date{\\today}\n\n",
        "\\begin{document}\n",
        "\\maketitle\n\n"
    ]
    file.writelines(preambulo)


# Función para escribir el cierre del archivo LaTeX
def escribir_cierre(file):
    file.write("\\end{document}\n")

# Función para generar una sección en el archivo LaTeX
def escribir_seccion(file, nombre_seccion, tipo_seccion="section"):
    file.write(f"\\{tipo_seccion}{{{nombre_seccion.replace('_', ' ')}}}\n")

# Función para agregar una imagen como figura al archivo LaTeX
def agregar_figura(file, image_path, caption_name, label_name):
    file.write("\\begin{figure}[H]\n")
    file.write(f"    \\centering\n")
    file.write(f"    \\includegraphics[width=1\\textwidth]{{{image_path}}}\n")
    file.write(f"    \\caption{{{caption_name}}}\n")
    file.write(f"    \\label{{{label_name}}}\n")
    file.write("\\end{figure}\n\n")

# Función para procesar gráficos en la carpeta "Graphics"
def procesar_carpeta_graphics(file, subfolder):
    print(f"Procesando gráficos en la carpeta: {subfolder}")
    subsubfolders = recorrer_subcarpetas(subfolder)
    
    # Variable para controlar si se ha creado la subsección principal para la carpeta
    subseccion_principal_creada = False

    for subsubfolder in subsubfolders:
        subsubfolder_name = os.path.basename(subsubfolder)

        # Extraer el nombre de la carpeta principal para verificar las subsecciones personalizadas
        carpeta_principal = os.path.basename(os.path.dirname(subsubfolder))

        # Si la carpeta principal coincide con alguna en el diccionario y aún no se ha creado la subsección principal
        if carpeta_principal in subsecciones_personalizadas and not subseccion_principal_creada:
            # Escribir la subsección personalizada
            escribir_seccion(file, subsecciones_personalizadas[carpeta_principal], tipo_seccion="subsection")
            subseccion_principal_creada = True  # Marcar que se ha creado la subsección principal

        # Listar los archivos PNG en la sub-subcarpeta
        png_files = [f for f in os.listdir(subsubfolder) if f.endswith('.png')]

        # Si no hay archivos PNG, continuar a la siguiente subcarpeta
        if not png_files:
            continue

        # Variable para almacenar el nombre completo de la triada correspondiente a esta subcarpeta
        nombre_triada = None

        # Verificar si alguno de los códigos en triad_names está en el nombre de la subcarpeta actual
        for code, name in triad_names.items():
            if code in subsubfolder_name:
                nombre_triada = name
                break

        # Crear una subsección con el nombre completo de la triada si se encontró
        if nombre_triada:
            escribir_seccion(file, nombre_triada, tipo_seccion="subsubsection")

        # Iterar sobre cada archivo PNG y agregarlo al documento LaTeX
        for png in png_files:
            if "MEI_NEGRA" in png:
                print(f"Excluyendo archivo: {png}")
                continue
            
            image_path = os.path.join(subsubfolder, png).replace(os.sep, '/')
            
            # Crear nombre y etiqueta de la figura sin caracteres especiales
            caption_name = f"Comparación: {os.path.splitext(png)[0].replace('_', ' ')}"
            label_name = f"fig:{os.path.splitext(png)[0]}"

            # Escribir la figura en el archivo LaTeX
            agregar_figura(file, image_path, caption_name, label_name)


# Función para procesar gráficos en la carpeta "Moduli Comparisons"
def procesar_carpeta_moduli_comparisons(file, subfolder):
    print(f"Procesando gráficos en la carpeta: {subfolder}")
    # Listar los archivos PNG en la carpeta directamente
    png_files = [f for f in os.listdir(subfolder) if f.endswith('.png')]

    # Iterar sobre cada archivo PNG y agregarlo al documento LaTeX
    for png in png_files:
        image_path = os.path.join(subfolder, png).replace(os.sep, '/')
        
        # Crear nombre y etiqueta de la figura sin caracteres especiales
        caption_name = f"Comparación Moduli: {os.path.splitext(png)[0].replace('_', ' ')}"
        label_name = f"fig:{os.path.splitext(png)[0]}"

        # Escribir la figura en el archivo LaTeX
        agregar_figura(file, image_path, caption_name, label_name)

# Función para procesar gráficos en la carpeta "Global Comparison"
def procesar_carpeta_global_comparisons(file, subfolder):
    print(f"Procesando gráficos en la carpeta: {subfolder}")
    
    # Ruta para la carpeta "Global Comparison"
    global_comparison_path = os.path.join(data_analysis_folder, "Global Comparisons")
    
    # Verificar si existe la carpeta "Global Comparison"
    if not os.path.exists(global_comparison_path):
        print(f"La carpeta 'Global Comparison' no existe en {subfolder}")
        return
    
    # Abrir el entorno de documento LaTeX
    file.write("\\section{Comparaciones Globales}\n")
    
    # Dimensiones para ajustar 4 imágenes en una página
    max_width = 0.45  # Porcentaje del ancho de la página
    max_height = 0.4  # Porcentaje del alto de la página ajustado para dos imágenes por fila
    
    # Iterar sobre las subcarpetas "M" y "V" dentro de "Global Comparison"
    for shift in ["M", "V"]:
        shift_path = os.path.join(global_comparison_path, shift)
        
        # Verificar si la subcarpeta existe
        if os.path.exists(shift_path):
            print(f"Procesando gráficos en la subcarpeta: {shift_path}")
            
            # Listar los archivos PNG en la subcarpeta
            png_files = [f for f in os.listdir(shift_path) if f.endswith('.png')]
            
            # Agrupar las imágenes de 4 en 4
            for i in range(0, len(png_files), 4):
                group_files = png_files[i:i+4]
                
                # Abrir un entorno figure
                file.write("\\begin{figure}[H]\n")
                file.write("\\centering\n")
                
                # Iterar sobre cada archivo PNG y agregarlo como subfigura
                for j, png in enumerate(group_files):
                    image_path = os.path.join(shift_path, png).replace(os.sep, '/')
                    caption_name = f"Comparación Global ({shift}): {os.path.splitext(png)[0].replace('_', ' ')}"
                    
                    # Alternar alineación para izquierda y derecha
                    alignment = "\\hfill" if j % 2 == 1 else ""
                    file.write(f"\\begin{{minipage}}[t]{{{max_width}\\textwidth}}{alignment}\n")
                    file.write(f"    \\includegraphics[width=\\textwidth,height={max_height}\\textheight,keepaspectratio]{{{image_path}}}\n")
                    file.write(f"    \\caption{{{caption_name}}}\n")
                    file.write(f"\\end{{minipage}}\n")
                    
                    # Salto de línea después de cada par de imágenes
                    if (j + 1) % 2 == 0 and j != len(group_files) - 1:
                        file.write("\\\\\n")
                
                # Cerrar el entorno figure
                file.write("\\end{figure}\n")
        else:
            print(f"La subcarpeta '{shift}' no existe en {global_comparison_path}")


def agregar_resultados_consolidados(file, root_folder):
    """
    Agrega el contenido del archivo Consolidated_Results.txt al documento LaTeX en un formato profesional.
    
    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - root_folder: directorio raíz que contiene el archivo Consolidated_Results.txt.
    """
    consolidated_file_path = os.path.join(root_folder, "Consolidated_Results.txt")
    
    if not os.path.exists(consolidated_file_path):
        print(f"El archivo {consolidated_file_path} no existe.")
        file.write("\\section{Topological Data Analysis}\n")
        file.write("No se encontró el archivo Consolidated_Results.txt.\n")
        return

    try:
        file.write("\\section{Topological Data Analysis}\n")
        with open(consolidated_file_path, 'r', encoding='ISO-8859-1') as consolidated_file:
            current_lab = None
            current_shift = None
            
            for line in consolidated_file:
                line = line.strip()
                
                # Identificar laboratorios
                if line.startswith("Laboratorio:"):
                    if current_lab:
                        file.write("\\end{verbatim}\n")  # Cerrar bloque anterior
                    current_lab = line.split(":", 1)[1].strip()
                    file.write(f"\\subsection*{{Laboratorio: {current_lab}}}\n")
                    file.write("\\begin{verbatim}\n")
                
                # Identificar turnos
                elif line.startswith("Turno:"):
                    if current_shift:
                        file.write("\\end{verbatim}\n")  # Cerrar bloque anterior
                    current_shift = line.split(":", 1)[1].strip()
                    file.write(f"\\subsubsection*{{Turno: {current_shift}}}\n")
                    file.write("\\begin{verbatim}\n")
                
                # Escribir contenido normal
                else:
                    file.write(line + "\n")
            
            # Cerrar el último bloque verbatim
            if current_lab or current_shift:
                file.write("\\end{verbatim}\n")
    except Exception as e:
        print(f"Error al leer el archivo {consolidated_file_path}: {e}")
        file.write("\\section{Resultados Consolidados}\n")
        file.write(f"Error al leer el archivo Consolidated_Results.txt: {e}\n")


def agregar_times_series_curl(file, root_folder, labs):
    """
    Agrega imágenes con sufijo 'time_series_full_comparison.png' al documento LaTeX 
    en la sección 'Times Series Curl'.

    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - root_folder: directorio raíz que contiene las carpetas.
    - labs: lista de laboratorios (e.g., ["Lab_Betta", "Lab_Gamma"]).
    """
    shifts = ["M", "V"]  # Turnos: Matutino (M) y Vespertino (V)
    date = os.path.basename(root_folder)  # Extraer la fecha del root_folder

    file.write("\\section{Times Series Curl}\n")

    for lab in labs:
        file.write(f"\\subsection{{\\textit{{{lab}}}}}\n")
        for shift in shifts:
            shift_name = "Matutino" if shift == 'M' else "Vespertino"
            output_folder = os.path.join(
                root_folder, 
                f"{date} - {lab}", 
                f"{date}.{shift} - {lab}", 
                "Data Analysis", 
                "Topological Data Analysis"
            )

            if os.path.exists(output_folder):
                # Agregar subsección para el turno
                file.write(f"\\subsubsection{{Turno {shift_name}}}\n")

                # Buscar imágenes con el sufijo 'time_series_full_comparison.png'
                images = [f for f in os.listdir(output_folder) if f.endswith("time_series_full_comparison.png")]
                if not images:
                    file.write("No se encontraron imágenes de comparación de series temporales.\n")
                    continue

                # Incluir cada imagen en el documento
                for image in images:
                    image_path = os.path.join(output_folder, image).replace(os.sep, '/')
                    caption = f"Comparación completa de series temporales: {os.path.splitext(image)[0].replace('_', ' ').capitalize()}"
                    label = f"fig:time_series_{lab}_{shift}_{os.path.splitext(image)[0]}"

                    file.write("\\begin{figure}[H]\n")
                    file.write("    \\centering\n")
                    file.write(f"    \\includegraphics[width=0.9\\textwidth]{{{image_path}}}\n")
                    file.write(f"    \\caption{{{caption}}}\n")
                    file.write(f"    \\label{{{label}}}\n")
                    file.write("\\end{figure}\n\n")
            else:
                # Indicar que no se encontró la carpeta correspondiente
                file.write(f"No se encontró la carpeta para Turno {shift_name}.\n")

def agregar_diferencias_tiempos(file, root_folder, labs):
    """
    Agrega una sección "Diferencias de Tiempos" al documento LaTeX,
    incluyendo imágenes de cada color desde las carpetas en "TimePlots".

    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - root_folder: directorio raíz que contiene las carpetas.
    - labs: lista de laboratorios (e.g., ["Lab_Betta", "Lab_Gamma"]).
    """
    # Crear la sección principal
    file.write("\\section{Diferencias de Tiempos}\n")

    for lab in labs:
        file.write(f"\\subsection{{\\textit{{{lab}}}}}\n")
        
        # Ruta al directorio TimePlots para el laboratorio actual
        timeplots_folder = os.path.join(root_folder, "Data Analysis", lab, "TimePlots")

        if os.path.exists(timeplots_folder):
            # Iterar sobre las carpetas de colores dentro de TimePlots
            color_folders = [f for f in os.listdir(timeplots_folder) if os.path.isdir(os.path.join(timeplots_folder, f))]

            for color_folder in color_folders:
                color_path = os.path.join(timeplots_folder, color_folder)
                images = [f for f in os.listdir(color_path) if f.endswith('.png')]

                if not images:
                    continue  # Si no hay imágenes, pasar al siguiente color

                # Crear una subsección para el color
                file.write(f"\\subsubsection{{Color {color_folder.capitalize()}}}\n")

                for image in images:
                    image_path = os.path.join(color_path, image).replace(os.sep, '/')
                    caption = f"Diferencias de tiempos: {os.path.splitext(image)[0].replace('_', ' ').capitalize()}"
                    label = f"fig:diferencias_tiempos_{lab}_{color_folder}_{os.path.splitext(image)[0]}"

                    # Incluir la imagen en el documento LaTeX
                    file.write("\\begin{figure}[H]\n")
                    file.write(f"    \\centering\n")
                    file.write(f"    \\includegraphics[width=0.9\\textwidth]{{{image_path}}}\n")
                    file.write(f"    \\caption{{{caption}}}\n")
                    file.write(f"    \\label{{{label}}}\n")
                    file.write("\\end{figure}\n\n")
        else:
            file.write(f"No se encontró la carpeta TimePlots para {lab}.\n")

def agregar_diagramas_persistencia(file, root_folder, labs):
    """
    Agrega imágenes con sufijo 'Comparative_Persistence.png' al documento LaTeX 
    en la sección 'Persistence Diagrams'.

    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - root_folder: directorio raíz que contiene las carpetas.
    - labs: lista de laboratorios (e.g., ["Lab_Betta", "Lab_Gamma"]).
    """
    shifts = ["M", "V"]  # Turnos: Matutino (M) y Vespertino (V)
    date = os.path.basename(root_folder)  # Extraer la fecha del root_folder

    file.write("\\section{Persistence Diagrams}\n")

    for lab in labs:
        file.write(f"\\subsection{{\\textit{{{lab}}}}}\n")
        for shift in shifts:
            shift_name = "Matutino" if shift == 'M' else "Vespertino"
            output_folder = os.path.join(
                root_folder, 
                f"{date} - {lab}", 
                f"{date}.{shift} - {lab}", 
                "Data Analysis", 
                "Topological Data Analysis"
            )

            if os.path.exists(output_folder):
                # Agregar subsección para el turno
                file.write(f"\\subsubsection{{Turno {shift_name}}}\n")

                # Buscar imágenes con el sufijo 'Comparative_Persistence.png'
                images = [f for f in os.listdir(output_folder) if f.endswith("Comparative_Persistence.png")]
                if not images:
                    file.write("No se encontraron diagramas de persistencia comparativos.\n")
                    continue

                # Incluir cada imagen en el documento
                for image in images:
                    image_path = os.path.join(output_folder, image).replace(os.sep, '/')
                    caption = f"Diagrama de persistencia comparativo: {os.path.splitext(image)[0].replace('_', ' ').capitalize()}"
                    label = f"fig:persistence_diagram_{lab}_{shift}_{os.path.splitext(image)[0]}"

                    file.write("\\begin{figure}[H]\n")
                    file.write("    \\centering\n")
                    file.write(f"    \\includegraphics[width=0.9\\textwidth]{{{image_path}}}\n")
                    file.write(f"    \\caption{{{caption}}}\n")
                    file.write(f"    \\label{{{label}}}\n")
                    file.write("\\end{figure}\n\n")
            else:
                # Indicar que no se encontró la carpeta correspondiente
                file.write(f"No se encontró la carpeta para Turno {shift_name}.\n")



# Función principal para procesar las subcarpetas en "Data Analysis"
def procesar_subcarpetas(data_analysis_folder, main_latex_file):
    try:
        with open(main_latex_file, 'w', encoding='utf-8') as file:
            # Escribir el preámbulo del archivo LaTeX
            escribir_preambulo(file, root_folder)

            generate_latex_statistics(file, root_folder, labs)
            
            procesar_comparison_times_series(file, labs)
            
            procesar_3d_curl(file, root_folder, "M")
            
            add_gravity_behavior_section(file, root_folder, labs)

            # Recorrer las subcarpetas de nivel superior dentro de "Data Analysis"
            subcarpetas_nivel_superior = [os.path.join(data_analysis_folder, folder) for folder in os.listdir(data_analysis_folder)
                                          if os.path.isdir(os.path.join(data_analysis_folder, folder))]

            # Cambiar el orden de las subsecciones:
            # 1. Global Comparisons
            # 3. Graphics
            for subfolder in subcarpetas_nivel_superior:
                subfolder_name = os.path.basename(subfolder)

                # Procesar primero la carpeta "Global Comparisons"
                if subfolder_name == "Global Comparisons":
                    escribir_seccion(file, subfolder_name)
                    procesar_carpeta_global_comparisons(file, subfolder)

            for subfolder in subcarpetas_nivel_superior:
                subfolder_name = os.path.basename(subfolder)

                # Procesar finalmente la carpeta "Graphics"
                if subfolder_name == "Graphics":
                    escribir_seccion(file, subfolder_name)
                    procesar_carpeta_graphics(file, subfolder)
            
            agregar_resultados_consolidados(file, root_folder)        
            
            agregar_times_series_curl(file, root_folder, labs)  

            agregar_diferencias_tiempos(file, root_folder, labs)

            agregar_diagramas_persistencia(file, root_folder, labs)  

            # Finalizar el documento LaTeX
            escribir_cierre(file)
            print(f"Archivo LaTeX generado correctamente: {main_latex_file}")
    except Exception as e:
        print(f"Error al generar el archivo LaTeX: {e}")

# Directorio raíz y configuración de carpetas
data_analysis_folder = os.path.join(root_folder, "Data Analysis")
tex_folder = os.path.join(root_folder, "TeX")

# Crear la carpeta "TeX" si no existe
crear_carpeta(tex_folder)

# Nombre del archivo LaTeX principal
main_latex_file = os.path.join(tex_folder, "main.tex")

# Verificar si la carpeta "Data Analysis" existe y procesar
if os.path.exists(data_analysis_folder):
    procesar_subcarpetas(data_analysis_folder, main_latex_file)
else:
    print(f"La carpeta {data_analysis_folder} no existe.")

In [ ]:
import os
import numpy as np
from scipy.stats import kurtosis, skew, iqr

subsecciones_personalizadas = {
    "Sum_Absolute_Helicity_Comp": "Suma acumulada de valores absolutos de la Helicidad",
    "Sum_Enstrophy_Comp": "Suma acumulada de la Enstrofia",
    "Sum_Helicity_Comp": "Suma acumulada de la Helicidad"
}

def add_gravity_behavior_section(file, root_folder, labs):
    """
    Agrega una sección "Gravity Behavior" al documento LaTeX, accediendo a las imágenes desde la carpeta "Pesajes/graficas".
    
    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - root_folder: directorio raíz que contiene las carpetas.
    - labs: lista de laboratorios (e.g., ["Lab_Betta", "Lab_Gamma"]).
    """
    shifts = ["M", "V"]  # Turnos: M = Matutino, V = Vespertino
    date = os.path.basename(root_folder)  # Extraer la fecha desde el root_folder

    # Crear la sección principal
    file.write("\\section{Gravity Behavior}\n")

    for lab in labs:
        file.write(f"\\subsection{{Lab \\textit{{{lab}}}}}\n")
        for shift in shifts:
            # Construir la ruta de la carpeta de imágenes
            output_folder = os.path.join(root_folder, f"{date} - {lab}", f"{date}.{shift} - {lab}", "Pesajes", "graficas")
            
            if os.path.exists(output_folder):
                file.write(f"\\subsubsection{{Turno {'Matutino' if shift == 'M' else 'Vespertino'}}}\n")

                # Listar las imágenes PNG en la carpeta
                images = [f for f in os.listdir(output_folder) if f.endswith('.png')]
                if not images:
                    continue  # Si no hay imágenes, pasar al siguiente turno

                for image in images:
                    image_path = os.path.join(output_folder, image).replace(os.sep, '/')
                    caption = f"Gravity Behavior: {os.path.splitext(image)[0].replace('_', ' ').capitalize()}"
                    label = f"fig:gravity_behavior_{lab}_{shift}_{os.path.splitext(image)[0]}"

                    # Incluir la imagen en el documento LaTeX
                    file.write("\\begin{figure}[H]\n")
                    file.write(f"    \\centering\n")
                    file.write(f"    \\includegraphics[width=0.9\\textwidth]{{{image_path}}}\n")
                    file.write(f"    \\caption{{{caption}}}\n")
                    file.write(f"    \\label{{{label}}}\n")
                    file.write("\\end{figure}\n\n")

def add_time_differences_section(file, root_folder, labs):
    """
    Agrega una sección "Diferencias de Tiempos" al documento LaTeX,
    incluyendo imágenes de cada color desde las carpetas en "TimePlots".

    Parámetros:
    - file: archivo LaTeX abierto en modo escritura.
    - root_folder: directorio raíz que contiene las carpetas.
    - labs: lista de laboratorios (e.g., ["Lab_Betta", "Lab_Gamma"]).
    """
    # Crear la sección principal
    file.write("\\section{Diferencias de Tiempos}\n")

    for lab in labs:
        file.write(f"\\subsection{{\\textit{{{lab}}}}}\n")
        
        # Ruta al directorio TimePlots para el laboratorio actual
        timeplots_folder = os.path.join(root_folder, "Data Analysis", lab, "TimePlots")

        if os.path.exists(timeplots_folder):
            # Iterar sobre las carpetas de colores dentro de TimePlots
            color_folders = [f for f in os.listdir(timeplots_folder) if os.path.isdir(os.path.join(timeplots_folder, f))]

            for color_folder in color_folders:
                color_path = os.path.join(timeplots_folder, color_folder)
                images = [f for f in os.listdir(color_path) if f.endswith('.png')]

                if not images:
                    continue  # Si no hay imágenes, pasar al siguiente color

                # Crear una subsección para el color
                file.write(f"\\subsubsection{{Color {color_folder.capitalize()}}}\n")

                for image in images:
                    image_path = os.path.join(color_path, image).replace(os.sep, '/')
                    caption = f"Diferencias de tiempos: {os.path.splitext(image)[0].replace('_', ' ').capitalize()}"
                    label = f"fig:diferencias_tiempos_{lab}_{color_folder}_{os.path.splitext(image)[0]}"

                    # Incluir la imagen en el documento LaTeX
                    file.write("\\begin{figure}[H]\n")
                    file.write(f"    \\centering\n")
                    file.write(f"    \\includegraphics[width=0.9\\textwidth]{{{image_path}}}\n")
                    file.write(f"    \\caption{{{caption}}}\n")
                    file.write(f"    \\label{{{label}}}\n")
                    file.write("\\end{figure}\n\n")
        else:
            file.write(f"No se encontró la carpeta TimePlots para {lab}.\n")

def analyze_folder(root_folder):
    """
    Recorre una carpeta y recopila datos de archivos específicos.
    """
    data_per_file = {}
    for root, dirs, files in os.walk(root_folder):
        if 'Data Analysis' in dirs:
            dirs.remove('Data Analysis')

        for filename in files:
            if filename.endswith(("negra.txt", "roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt")):
                file_path = os.path.join(root, filename)
                timestamps, data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = (timestamps, data, file_path)
    return data_per_file

# Función para leer datos de los sensores
def read_sensor_data(filepath):
    """
    Lee datos de sensores desde un archivo.
    """
    data = []
    timestamps = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return timestamps, np.array(data).T

def crear_carpeta(ruta_carpeta):
    """
    Verifica y crea una carpeta si no existe.
    """
    try:
        os.makedirs(ruta_carpeta, exist_ok=True)
        print(f"Carpeta creada o ya existente: {ruta_carpeta}")
    except Exception as e:
        print(f"Error al crear la carpeta {ruta_carpeta}: {e}")

# Función principal para procesar las subcarpetas en "Data Analysis"
def procesar_subcarpetas(data_analysis_folder, main_latex_file):
    try:
        with open(main_latex_file, 'w', encoding='utf-8') as file:
            # Escribir el preámbulo del archivo LaTeX
            escribir_preambulo(file, root_folder)

            generate_latex_statistics(file, root_folder, labs)
            procesar_comparison_times_series(file, labs)
            procesar_3d_curl(file, root_folder, "M")
            add_gravity_behavior_section(file, root_folder, labs)
            add_time_differences_section(file, root_folder, labs)

            # Recorrer las subcarpetas de nivel superior dentro de "Data Analysis"
            subcarpetas_nivel_superior = [os.path.join(data_analysis_folder, folder) for folder in os.listdir(data_analysis_folder)
                                          if os.path.isdir(os.path.join(data_analysis_folder, folder))]

            # Cambiar el orden de las subsecciones:
            # 1. Global Comparisons
            # 3. Graphics
            for subfolder in subcarpetas_nivel_superior:
                subfolder_name = os.path.basename(subfolder)

                # Procesar primero la carpeta "Global Comparisons"
                if subfolder_name == "Global Comparisons":
                    escribir_seccion(file, subfolder_name)
                    procesar_carpeta_global_comparisons(file, subfolder)

            for subfolder in subcarpetas_nivel_superior:
                subfolder_name = os.path.basename(subfolder)

                # Procesar finalmente la carpeta "Graphics"
                if subfolder_name == "Graphics":
                    escribir_seccion(file, subfolder_name)
                    procesar_carpeta_graphics(file, subfolder)

            agregar_resultados_consolidados(file, root_folder)        
            agregar_times_series_curl(file, root_folder, labs)  
            agregar_diagramas_persistencia(file, root_folder, labs)  

            # Finalizar el documento LaTeX
            escribir_cierre(file)
            print(f"Archivo LaTeX generado correctamente: {main_latex_file}")
    except Exception as e:
        print(f"Error al generar el archivo LaTeX: {e}")

# Directorio raíz y configuración de carpetas
data_analysis_folder = os.path.join(root_folder, "Data Analysis")
tex_folder = os.path.join(root_folder, "TeX")

# Crear la carpeta "TeX" si no existe
crear_carpeta(tex_folder)

# Nombre del archivo LaTeX principal
main_latex_file = os.path.join(tex_folder, "main.tex")

# Verificar si la carpeta "Data Analysis" existe y procesar
if os.path.exists(data_analysis_folder):
    procesar_subcarpetas(data_analysis_folder, main_latex_file)
else:
    print(f"La carpeta {data_analysis_folder} no existe.")